### Imports and Set Flag

In [156]:
import os
import json
import pandas as pd
import polars as pl
import numpy as np

from tqdm import tqdm

#TODO: Only get metadata for MLP

### Environment Setup ###
IS_KAGGLE = False  # Flag to switch between environments

# Progress bar - only create if running locally
N_TEST_DAYS = 4   # Number of days in test set, purely to have a progress bar
progress_bar = None if IS_KAGGLE else tqdm(total=N_TEST_DAYS, desc='Processing days')

Processing days:   0%|          | 0/4 [00:05<?, ?it/s]


### Environment Dependent Path Setup

In [157]:
# Setup paths based on environment
if IS_KAGGLE:
    data_path = '/kaggle/input/jane-street-real-time-market-data-forecasting'
    trained_models_path = "/kaggle/input/janestreet-models/"
else:
    # For local testing
    import sys

    # Path to main project directory
    PROJ_DIR = os.path.dirname(os.getcwd())
    
    # Add paths to system path
    sys.path.append(os.path.join(PROJ_DIR, "jane-street-real-time-market-data-forecasting"))
    sys.path.append(os.path.join(PROJ_DIR, "training", "src", "utils"))
    
    # Set data path to our local test data
    data_path = os.path.join(os.getcwd(), "local_test_data")
    
    # Import local testing metrics
    from metrics import r2_score_weighted

import kaggle_evaluation.jane_street_inference_server

### Pre-processing

#### A global dictionary that initialises with the last seen values for the set we are about to see.

In [158]:
last_known_values = {
    (0, 'feature_00'): 2.827219,
    (0, 'feature_01'): -0.875262,
    (0, 'feature_02'): 2.520329,
    (0, 'feature_03'): 1.967673,
    (0, 'feature_04'): 0.516955,
    (0, 'feature_05'): 0.528624,
    (0, 'feature_06'): 0.759274,
    (0, 'feature_07'): 1.303269,
    (0, 'feature_08'): 0.172209,
    (0, 'feature_09'): 11.000000,
    (0, 'feature_10'): 7.000000,
    (0, 'feature_11'): 76.000000,
    (0, 'feature_12'): 0.422208,
    (0, 'feature_13'): 0.544176,
    (0, 'feature_14'): 0.088627,
    (0, 'feature_15'): -0.838364,
    (0, 'feature_16'): -0.385189,
    (0, 'feature_17'): -0.794437,
    (0, 'feature_18'): 1.732842,
    (0, 'feature_19'): 0.257682,
    (0, 'feature_20'): 1.515887,
    (0, 'feature_21'): -0.202102,
    (0, 'feature_22'): 1.285443,
    (0, 'feature_23'): 0.787999,
    (0, 'feature_24'): 1.921844,
    (0, 'feature_25'): 1.281180,
    (0, 'feature_26'): 1.223592,
    (0, 'feature_27'): 1.002706,
    (0, 'feature_28'): 0.587736,
    (0, 'feature_29'): -0.643336,
    (0, 'feature_30'): -0.943377,
    (0, 'feature_31'): -0.148149,
    (0, 'feature_32'): 3.395344,
    (0, 'feature_33'): -0.186680,
    (0, 'feature_34'): 3.202790,
    (0, 'feature_35'): 3.064410,
    (0, 'feature_36'): -1.320826,
    (0, 'feature_37'): -0.085313,
    (0, 'feature_38'): -0.115382,
    (0, 'feature_39'): 1.000890,
    (0, 'feature_40'): -0.303232,
    (0, 'feature_41'): -0.448312,
    (0, 'feature_42'): 1.107358,
    (0, 'feature_43'): -0.227307,
    (0, 'feature_44'): -0.179370,
    (0, 'feature_45'): 0.597232,
    (0, 'feature_46'): 0.092410,
    (0, 'feature_47'): 0.445950,
    (0, 'feature_48'): 0.340762,
    (0, 'feature_49'): 0.465917,
    (0, 'feature_50'): 0.630211,
    (0, 'feature_51'): -0.318139,
    (0, 'feature_52'): 0.144056,
    (0, 'feature_53'): 1.281280,
    (0, 'feature_54'): 0.335029,
    (0, 'feature_55'): 1.240806,
    (0, 'feature_56'): 0.046584,
    (0, 'feature_57'): 0.040414,
    (0, 'feature_58'): 0.506329,
    (0, 'feature_59'): 0.187320,
    (0, 'feature_60'): 0.798328,
    (0, 'feature_61'): -1.101531,
    (0, 'feature_62'): -0.469153,
    (0, 'feature_63'): -0.352623,
    (0, 'feature_64'): -0.381301,
    (0, 'feature_65'): 2.008002,
    (0, 'feature_66'): -0.013916,
    (0, 'feature_67'): 0.302435,
    (0, 'feature_68'): 0.128049,
    (0, 'feature_69'): -0.145918,
    (0, 'feature_70'): 0.417643,
    (0, 'feature_71'): 0.921223,
    (0, 'feature_72'): 0.501211,
    (0, 'feature_73'): -0.187231,
    (0, 'feature_74'): -0.121017,
    (0, 'feature_75'): -0.306457,
    (0, 'feature_76'): -0.204136,
    (0, 'feature_77'): -0.133185,
    (0, 'feature_78'): -0.165851,
    (1, 'feature_00'): 2.013694,
    (1, 'feature_01'): -0.490128,
    (1, 'feature_02'): 3.178985,
    (1, 'feature_03'): 2.103597,
    (1, 'feature_04'): 0.696693,
    (1, 'feature_05'): 0.468617,
    (1, 'feature_06'): 1.183813,
    (1, 'feature_07'): 1.272885,
    (1, 'feature_08'): 0.207234,
    (1, 'feature_09'): 11.000000,
    (1, 'feature_10'): 7.000000,
    (1, 'feature_11'): 76.000000,
    (1, 'feature_12'): 0.370844,
    (1, 'feature_13'): 0.051970,
    (1, 'feature_14'): -0.090220,
    (1, 'feature_15'): -0.582837,
    (1, 'feature_16'): -0.689992,
    (1, 'feature_17'): -0.734519,
    (1, 'feature_18'): 2.304565,
    (1, 'feature_19'): -0.202955,
    (1, 'feature_20'): 1.280485,
    (1, 'feature_21'): -0.024357,
    (1, 'feature_22'): 0.936136,
    (1, 'feature_23'): 1.288942,
    (1, 'feature_24'): 0.845199,
    (1, 'feature_25'): 0.516642,
    (1, 'feature_26'): -1.466948,
    (1, 'feature_27'): 0.035089,
    (1, 'feature_28'): 1.195631,
    (1, 'feature_29'): -0.671953,
    (1, 'feature_30'): -0.648468,
    (1, 'feature_31'): -0.030753,
    (1, 'feature_32'): 2.830256,
    (1, 'feature_33'): -0.897922,
    (1, 'feature_34'): 3.275199,
    (1, 'feature_35'): 2.959840,
    (1, 'feature_36'): 0.552437,
    (1, 'feature_37'): 0.137097,
    (1, 'feature_38'): 0.031978,
    (1, 'feature_39'): 0.815463,
    (1, 'feature_40'): 0.005789,
    (1, 'feature_41'): 0.219333,
    (1, 'feature_42'): 0.101030,
    (1, 'feature_43'): 0.097533,
    (1, 'feature_44'): 0.583423,
    (1, 'feature_45'): 0.068223,
    (1, 'feature_46'): -1.218917,
    (1, 'feature_47'): -0.207014,
    (1, 'feature_48'): 0.415156,
    (1, 'feature_49'): 0.497565,
    (1, 'feature_50'): -0.236869,
    (1, 'feature_51'): 0.035107,
    (1, 'feature_52'): 0.138957,
    (1, 'feature_53'): 0.755119,
    (1, 'feature_54'): 0.385377,
    (1, 'feature_55'): 0.869776,
    (1, 'feature_56'): -0.279790,
    (1, 'feature_57'): -1.001044,
    (1, 'feature_58'): -0.160609,
    (1, 'feature_59'): 0.930379,
    (1, 'feature_60'): 0.504500,
    (1, 'feature_61'): -1.101531,
    (1, 'feature_62'): -0.327727,
    (1, 'feature_63'): -0.396467,
    (1, 'feature_64'): -0.365352,
    (1, 'feature_65'): 2.017632,
    (1, 'feature_66'): 0.305675,
    (1, 'feature_67'): 0.293498,
    (1, 'feature_68'): -0.268186,
    (1, 'feature_69'): -0.187959,
    (1, 'feature_70'): 0.317629,
    (1, 'feature_71'): 0.369344,
    (1, 'feature_72'): 0.066589,
    (1, 'feature_73'): -0.196612,
    (1, 'feature_74'): -0.270597,
    (1, 'feature_75'): -0.270793,
    (1, 'feature_76'): -0.266943,
    (1, 'feature_77'): -0.038692,
    (1, 'feature_78'): -0.047244,
    (2, 'feature_00'): 2.314176,
    (2, 'feature_01'): -0.850854,
    (2, 'feature_02'): 2.832688,
    (2, 'feature_03'): 3.151216,
    (2, 'feature_04'): 0.874473,
    (2, 'feature_05'): 0.749497,
    (2, 'feature_06'): 1.309507,
    (2, 'feature_07'): 1.695813,
    (2, 'feature_08'): 0.416761,
    (2, 'feature_09'): 81.000000,
    (2, 'feature_10'): 2.000000,
    (2, 'feature_11'): 59.000000,
    (2, 'feature_12'): 0.168929,
    (2, 'feature_13'): 0.052472,
    (2, 'feature_14'): -0.027308,
    (2, 'feature_15'): -0.648500,
    (2, 'feature_16'): -0.657262,
    (2, 'feature_17'): -0.508287,
    (2, 'feature_18'): 2.501936,
    (2, 'feature_19'): 1.922898,
    (2, 'feature_20'): 0.080837,
    (2, 'feature_21'): -0.174716,
    (2, 'feature_22'): 1.078999,
    (2, 'feature_23'): 0.448147,
    (2, 'feature_24'): 0.117895,
    (2, 'feature_25'): -0.680010,
    (2, 'feature_26'): 0.717676,
    (2, 'feature_27'): 0.835950,
    (2, 'feature_28'): 0.450227,
    (2, 'feature_29'): -0.789566,
    (2, 'feature_30'): -0.666649,
    (2, 'feature_31'): -0.171650,
    (2, 'feature_32'): 2.721324,
    (2, 'feature_33'): -0.563602,
    (2, 'feature_34'): 3.211753,
    (2, 'feature_35'): 3.106503,
    (2, 'feature_36'): -0.132089,
    (2, 'feature_37'): -0.823983,
    (2, 'feature_38'): -0.930735,
    (2, 'feature_39'): 1.859080,
    (2, 'feature_40'): 0.186093,
    (2, 'feature_41'): 1.562226,
    (2, 'feature_42'): 0.660685,
    (2, 'feature_43'): -0.040951,
    (2, 'feature_44'): 0.225281,
    (2, 'feature_45'): 0.552553,
    (2, 'feature_46'): -0.271673,
    (2, 'feature_47'): 0.627877,
    (2, 'feature_48'): 0.597258,
    (2, 'feature_49'): 1.340464,
    (2, 'feature_50'): 1.130593,
    (2, 'feature_51'): 0.331053,
    (2, 'feature_52'): 1.519897,
    (2, 'feature_53'): 1.935962,
    (2, 'feature_54'): 0.714714,
    (2, 'feature_55'): 1.215060,
    (2, 'feature_56'): -0.104940,
    (2, 'feature_57'): 0.123311,
    (2, 'feature_58'): 1.147459,
    (2, 'feature_59'): 1.334871,
    (2, 'feature_60'): 2.482095,
    (2, 'feature_61'): -1.101531,
    (2, 'feature_62'): -0.716749,
    (2, 'feature_63'): -0.415525,
    (2, 'feature_64'): -0.538249,
    (2, 'feature_65'): 2.334168,
    (2, 'feature_66'): 1.310023,
    (2, 'feature_67'): 0.216282,
    (2, 'feature_68'): -0.135751,
    (2, 'feature_69'): -0.218004,
    (2, 'feature_70'): 0.176190,
    (2, 'feature_71'): 0.318117,
    (2, 'feature_72'): 0.162799,
    (2, 'feature_73'): -0.315847,
    (2, 'feature_74'): -0.277738,
    (2, 'feature_75'): -0.253755,
    (2, 'feature_76'): -0.143074,
    (2, 'feature_77'): -0.240364,
    (2, 'feature_78'): -0.196238,
    (3, 'feature_00'): 2.889366,
    (3, 'feature_01'): -0.726483,
    (3, 'feature_02'): 2.579133,
    (3, 'feature_03'): 2.185707,
    (3, 'feature_04'): 0.531747,
    (3, 'feature_05'): 0.841699,
    (3, 'feature_06'): 1.287534,
    (3, 'feature_07'): 1.267083,
    (3, 'feature_08'): 0.283362,
    (3, 'feature_09'): 4.000000,
    (3, 'feature_10'): 3.000000,
    (3, 'feature_11'): 11.000000,
    (3, 'feature_12'): 0.755318,
    (3, 'feature_13'): 0.804561,
    (3, 'feature_14'): 0.509242,
    (3, 'feature_15'): -0.540194,
    (3, 'feature_16'): -0.715664,
    (3, 'feature_17'): -0.740786,
    (3, 'feature_18'): 2.059797,
    (3, 'feature_19'): 1.254693,
    (3, 'feature_20'): -0.185432,
    (3, 'feature_21'): -0.032784,
    (3, 'feature_22'): 0.513848,
    (3, 'feature_23'): 0.911167,
    (3, 'feature_24'): 0.139900,
    (3, 'feature_25'): -0.494777,
    (3, 'feature_26'): -0.010124,
    (3, 'feature_27'): -0.061895,
    (3, 'feature_28'): -0.303735,
    (3, 'feature_29'): -0.851368,
    (3, 'feature_30'): -0.481167,
    (3, 'feature_31'): -0.043368,
    (3, 'feature_32'): 3.517096,
    (3, 'feature_33'): -0.505588,
    (3, 'feature_34'): 3.440826,
    (3, 'feature_35'): 3.549078,
    (3, 'feature_36'): -0.907730,
    (3, 'feature_37'): -0.038709,
    (3, 'feature_38'): -0.080895,
    (3, 'feature_39'): 0.032243,
    (3, 'feature_40'): -1.005228,
    (3, 'feature_41'): -0.191002,
    (3, 'feature_42'): 0.368445,
    (3, 'feature_43'): -0.124435,
    (3, 'feature_44'): -0.635766,
    (3, 'feature_45'): -0.437964,
    (3, 'feature_46'): -1.093596,
    (3, 'feature_47'): -0.441981,
    (3, 'feature_48'): -0.463793,
    (3, 'feature_49'): 0.019661,
    (3, 'feature_50'): 0.753141,
    (3, 'feature_51'): -0.215869,
    (3, 'feature_52'): 0.459585,
    (3, 'feature_53'): 0.446787,
    (3, 'feature_54'): 0.392017,
    (3, 'feature_55'): -0.500262,
    (3, 'feature_56'): -0.185416,
    (3, 'feature_57'): -0.133476,
    (3, 'feature_58'): -0.183937,
    (3, 'feature_59'): -0.330193,
    (3, 'feature_60'): 0.302970,
    (3, 'feature_61'): -1.101531,
    (3, 'feature_62'): -0.418620,
    (3, 'feature_63'): -0.283908,
    (3, 'feature_64'): -0.273953,
    (3, 'feature_65'): 2.328944,
    (3, 'feature_66'): 1.595143,
    (3, 'feature_67'): 0.995962,
    (3, 'feature_68'): 0.448719,
    (3, 'feature_69'): 0.408619,
    (3, 'feature_70'): 0.840456,
    (3, 'feature_71'): 1.014184,
    (3, 'feature_72'): 0.961242,
    (3, 'feature_73'): -0.149760,
    (3, 'feature_74'): -0.133092,
    (3, 'feature_75'): -0.230151,
    (3, 'feature_76'): -0.301071,
    (3, 'feature_77'): -0.193645,
    (3, 'feature_78'): -0.233608,
    (4, 'feature_00'): 2.408940,
    (4, 'feature_01'): -0.404262,
    (4, 'feature_02'): 2.389869,
    (4, 'feature_03'): 2.148308,
    (4, 'feature_04'): 0.690830,
    (4, 'feature_05'): 0.343250,
    (4, 'feature_06'): 0.311630,
    (4, 'feature_07'): 1.249690,
    (4, 'feature_08'): 0.146407,
    (4, 'feature_09'): 15.000000,
    (4, 'feature_10'): 1.000000,
    (4, 'feature_11'): 9.000000,
    (4, 'feature_12'): -0.072712,
    (4, 'feature_13'): -0.064810,
    (4, 'feature_14'): -0.020689,
    (4, 'feature_15'): -0.874757,
    (4, 'feature_16'): -0.834729,
    (4, 'feature_17'): -0.811819,
    (4, 'feature_18'): 1.037543,
    (4, 'feature_19'): 0.086536,
    (4, 'feature_20'): -1.308994,
    (4, 'feature_21'): 0.313271,
    (4, 'feature_22'): 0.505732,
    (4, 'feature_23'): 0.555471,
    (4, 'feature_24'): 3.158962,
    (4, 'feature_25'): 2.240473,
    (4, 'feature_26'): -1.004671,
    (4, 'feature_27'): -1.344259,
    (4, 'feature_28'): -0.679483,
    (4, 'feature_29'): -0.745327,
    (4, 'feature_30'): -1.006627,
    (4, 'feature_31'): 0.354184,
    (4, 'feature_32'): 2.208168,
    (4, 'feature_33'): 1.294857,
    (4, 'feature_34'): 3.634552,
    (4, 'feature_35'): 3.438786,
    (4, 'feature_36'): 0.012311,
    (4, 'feature_37'): -0.013609,
    (4, 'feature_38'): -0.072169,
    (4, 'feature_39'): -0.348775,
    (4, 'feature_40'): -0.735364,
    (4, 'feature_41'): -0.924002,
    (4, 'feature_42'): 0.878732,
    (4, 'feature_43'): -0.431216,
    (4, 'feature_44'): -0.151823,
    (4, 'feature_45'): -1.206081,
    (4, 'feature_46'): 0.462461,
    (4, 'feature_47'): 0.069101,
    (4, 'feature_48'): -0.126983,
    (4, 'feature_49'): 0.042352,
    (4, 'feature_50'): -0.803048,
    (4, 'feature_51'): -0.456282,
    (4, 'feature_52'): -0.605650,
    (4, 'feature_53'): 1.890554,
    (4, 'feature_54'): 0.568981,
    (4, 'feature_55'): 1.878319,
    (4, 'feature_56'): -0.975481,
    (4, 'feature_57'): 0.742094,
    (4, 'feature_58'): 0.031067,
    (4, 'feature_59'): 0.111810,
    (4, 'feature_60'): 0.128299,
    (4, 'feature_61'): -1.101531,
    (4, 'feature_62'): -0.483739,
    (4, 'feature_63'): -0.429612,
    (4, 'feature_64'): -0.301022,
    (4, 'feature_65'): 2.297882,
    (4, 'feature_66'): 1.826284,
    (4, 'feature_67'): -0.212971,
    (4, 'feature_68'): -0.267840,
    (4, 'feature_69'): -0.210466,
    (4, 'feature_70'): -0.098667,
    (4, 'feature_71'): 0.125621,
    (4, 'feature_72'): 0.120593,
    (4, 'feature_73'): -0.228734,
    (4, 'feature_74'): -0.174740,
    (4, 'feature_75'): -0.229592,
    (4, 'feature_76'): -0.219182,
    (4, 'feature_77'): -0.195354,
    (4, 'feature_78'): -0.200176,
    (5, 'feature_00'): 2.684347,
    (5, 'feature_01'): -0.872678,
    (5, 'feature_02'): 2.311741,
    (5, 'feature_03'): 2.779893,
    (5, 'feature_04'): 0.140192,
    (5, 'feature_05'): 0.686878,
    (5, 'feature_06'): 1.602279,
    (5, 'feature_07'): 2.404034,
    (5, 'feature_08'): 0.296327,
    (5, 'feature_09'): 2.000000,
    (5, 'feature_10'): 10.000000,
    (5, 'feature_11'): 171.000000,
    (5, 'feature_12'): 0.222310,
    (5, 'feature_13'): 0.197121,
    (5, 'feature_14'): 0.471111,
    (5, 'feature_15'): -0.453560,
    (5, 'feature_16'): -0.632421,
    (5, 'feature_17'): -0.532357,
    (5, 'feature_18'): 0.043765,
    (5, 'feature_19'): 1.513533,
    (5, 'feature_20'): 0.230665,
    (5, 'feature_21'): -0.057268,
    (5, 'feature_22'): 1.490816,
    (5, 'feature_23'): 0.464915,
    (5, 'feature_24'): 0.102703,
    (5, 'feature_25'): -0.363315,
    (5, 'feature_26'): -1.127638,
    (5, 'feature_27'): 0.375466,
    (5, 'feature_28'): 1.059192,
    (5, 'feature_29'): -0.910429,
    (5, 'feature_30'): -0.811368,
    (5, 'feature_31'): -0.074974,
    (5, 'feature_32'): 2.620925,
    (5, 'feature_33'): -0.567229,
    (5, 'feature_34'): 3.034872,
    (5, 'feature_35'): 3.456723,
    (5, 'feature_36'): -0.767637,
    (5, 'feature_37'): -0.276754,
    (5, 'feature_38'): -0.367786,
    (5, 'feature_39'): -0.161150,
    (5, 'feature_40'): -0.134094,
    (5, 'feature_41'): 0.625632,
    (5, 'feature_42'): -0.969785,
    (5, 'feature_43'): -0.420579,
    (5, 'feature_44'): -0.415820,
    (5, 'feature_45'): 0.259518,
    (5, 'feature_46'): -0.901826,
    (5, 'feature_47'): -0.368840,
    (5, 'feature_48'): 0.027881,
    (5, 'feature_49'): -0.135576,
    (5, 'feature_50'): 0.065302,
    (5, 'feature_51'): 0.389036,
    (5, 'feature_52'): 1.217373,
    (5, 'feature_53'): -1.434994,
    (5, 'feature_54'): -1.181192,
    (5, 'feature_55'): 0.627334,
    (5, 'feature_56'): -0.605111,
    (5, 'feature_57'): -0.100596,
    (5, 'feature_58'): -0.257102,
    (5, 'feature_59'): 0.961842,
    (5, 'feature_60'): 0.170854,
    (5, 'feature_61'): -1.101531,
    (5, 'feature_62'): -0.466306,
    (5, 'feature_63'): -0.422564,
    (5, 'feature_64'): -0.472585,
    (5, 'feature_65'): 2.372541,
    (5, 'feature_66'): 2.033003,
    (5, 'feature_67'): 0.292430,
    (5, 'feature_68'): 0.082310,
    (5, 'feature_69'): 0.129773,
    (5, 'feature_70'): 0.480861,
    (5, 'feature_71'): 0.397526,
    (5, 'feature_72'): 0.567916,
    (5, 'feature_73'): 5.272311,
    (5, 'feature_74'): 3.405017,
    (5, 'feature_75'): 3.913973,
    (5, 'feature_76'): 3.298062,
    (5, 'feature_77'): 4.096728,
    (5, 'feature_78'): 3.959293,
    (6, 'feature_00'): 2.690066,
    (6, 'feature_01'): -0.511407,
    (6, 'feature_02'): 2.182443,
    (6, 'feature_03'): 1.976146,
    (6, 'feature_04'): 0.286973,
    (6, 'feature_05'): 0.807627,
    (6, 'feature_06'): 1.138151,
    (6, 'feature_07'): 2.002218,
    (6, 'feature_08'): 0.252900,
    (6, 'feature_09'): 12.000000,
    (6, 'feature_10'): 4.000000,
    (6, 'feature_11'): 34.000000,
    (6, 'feature_12'): 0.315203,
    (6, 'feature_13'): 1.052348,
    (6, 'feature_14'): 0.651208,
    (6, 'feature_15'): -0.635930,
    (6, 'feature_16'): -0.365874,
    (6, 'feature_17'): -0.563224,
    (6, 'feature_18'): 2.249499,
    (6, 'feature_19'): -1.677626,
    (6, 'feature_20'): -1.844685,
    (6, 'feature_21'): -0.338941,
    (6, 'feature_22'): -0.636572,
    (6, 'feature_23'): -0.619490,
    (6, 'feature_24'): 0.543353,
    (6, 'feature_25'): 0.000065,
    (6, 'feature_26'): -0.476890,
    (6, 'feature_27'): -0.148591,
    (6, 'feature_28'): -0.249568,
    (6, 'feature_29'): -0.815370,
    (6, 'feature_30'): -0.702343,
    (6, 'feature_31'): -0.277895,
    (6, 'feature_32'): 3.145638,
    (6, 'feature_33'): -0.414214,
    (6, 'feature_34'): 3.235271,
    (6, 'feature_35'): 3.221557,
    (6, 'feature_36'): -0.195740,
    (6, 'feature_37'): 0.345029,
    (6, 'feature_38'): 0.182916,
    (6, 'feature_39'): 0.836283,
    (6, 'feature_40'): -0.776329,
    (6, 'feature_41'): 0.710949,
    (6, 'feature_42'): 0.402781,
    (6, 'feature_43'): -1.430838,
    (6, 'feature_44'): -0.690622,
    (6, 'feature_45'): -0.101898,
    (6, 'feature_46'): -0.771390,
    (6, 'feature_47'): 0.773420,
    (6, 'feature_48'): -1.663328,
    (6, 'feature_49'): 0.716509,
    (6, 'feature_50'): 0.722813,
    (6, 'feature_51'): 2.156629,
    (6, 'feature_52'): 2.297484,
    (6, 'feature_53'): -0.342947,
    (6, 'feature_54'): -1.712906,
    (6, 'feature_55'): -1.104673,
    (6, 'feature_56'): 0.490489,
    (6, 'feature_57'): -0.799917,
    (6, 'feature_58'): 0.694747,
    (6, 'feature_59'): -0.936821,
    (6, 'feature_60'): 0.726885,
    (6, 'feature_61'): -1.101531,
    (6, 'feature_62'): -0.307937,
    (6, 'feature_63'): -0.016711,
    (6, 'feature_64'): -0.390561,
    (6, 'feature_65'): 2.788516,
    (6, 'feature_66'): -0.983793,
    (6, 'feature_67'): 0.246092,
    (6, 'feature_68'): 0.797856,
    (6, 'feature_69'): 0.172030,
    (6, 'feature_70'): 0.241918,
    (6, 'feature_71'): 1.770469,
    (6, 'feature_72'): 1.013370,
    (6, 'feature_73'): 0.221069,
    (6, 'feature_74'): 0.225807,
    (6, 'feature_75'): 0.229402,
    (6, 'feature_76'): 0.312497,
    (6, 'feature_77'): 0.285886,
    (6, 'feature_78'): 0.392722,
    (7, 'feature_00'): 2.356212,
    (7, 'feature_01'): -0.898497,
    (7, 'feature_02'): 2.090176,
    (7, 'feature_03'): 2.408094,
    (7, 'feature_04'): 0.271792,
    (7, 'feature_05'): 0.644885,
    (7, 'feature_06'): 0.770334,
    (7, 'feature_07'): 1.248405,
    (7, 'feature_08'): 0.267808,
    (7, 'feature_09'): 11.000000,
    (7, 'feature_10'): 7.000000,
    (7, 'feature_11'): 76.000000,
    (7, 'feature_12'): 1.048732,
    (7, 'feature_13'): 2.311911,
    (7, 'feature_14'): 0.966095,
    (7, 'feature_15'): -0.668570,
    (7, 'feature_16'): -0.435352,
    (7, 'feature_17'): -0.591820,
    (7, 'feature_18'): 1.851331,
    (7, 'feature_19'): -0.013700,
    (7, 'feature_20'): 1.344802,
    (7, 'feature_21'): -0.024406,
    (7, 'feature_22'): 1.906385,
    (7, 'feature_23'): 1.304772,
    (7, 'feature_24'): 0.979964,
    (7, 'feature_25'): 0.895315,
    (7, 'feature_26'): -0.235107,
    (7, 'feature_27'): 0.561495,
    (7, 'feature_28'): 0.374949,
    (7, 'feature_29'): -0.460062,
    (7, 'feature_30'): -0.599877,
    (7, 'feature_31'): -0.031231,
    (7, 'feature_32'): 3.318656,
    (7, 'feature_33'): -0.213651,
    (7, 'feature_34'): 2.913191,
    (7, 'feature_35'): 3.004750,
    (7, 'feature_36'): -0.470634,
    (7, 'feature_37'): 0.627877,
    (7, 'feature_38'): 0.340458,
    (7, 'feature_39'): -0.646081,
    (7, 'feature_40'): -0.453362,
    (7, 'feature_41'): 1.161974,
    (7, 'feature_42'): -0.351140,
    (7, 'feature_43'): -0.258424,
    (7, 'feature_44'): 0.304439,
    (7, 'feature_45'): 0.081930,
    (7, 'feature_46'): -0.390854,
    (7, 'feature_47'): -0.128900,
    (7, 'feature_48'): 1.190736,
    (7, 'feature_49'): 0.255926,
    (7, 'feature_50'): 0.180920,
    (7, 'feature_51'): -0.399398,
    (7, 'feature_52'): 0.280023,
    (7, 'feature_53'): -0.158125,
    (7, 'feature_54'): 0.744842,
    (7, 'feature_55'): 1.944974,
    (7, 'feature_56'): -0.686198,
    (7, 'feature_57'): -0.345362,
    (7, 'feature_58'): -0.275413,
    (7, 'feature_59'): 1.342141,
    (7, 'feature_60'): 0.256276,
    (7, 'feature_61'): -1.101531,
    (7, 'feature_62'): -0.229428,
    (7, 'feature_63'): -0.340171,
    (7, 'feature_64'): -0.460296,
    (7, 'feature_65'): 1.502340,
    (7, 'feature_66'): 0.655869,
    (7, 'feature_67'): 0.912502,
    (7, 'feature_68'): 1.371101,
    (7, 'feature_69'): 0.513698,
    (7, 'feature_70'): 0.662311,
    (7, 'feature_71'): 1.709862,
    (7, 'feature_72'): 1.652276,
    (7, 'feature_73'): -0.371009,
    (7, 'feature_74'): -0.370410,
    (7, 'feature_75'): -0.207106,
    (7, 'feature_76'): -0.308972,
    (7, 'feature_77'): -0.265729,
    (7, 'feature_78'): -0.386768,
    (8, 'feature_00'): 2.437362,
    (8, 'feature_01'): -0.797020,
    (8, 'feature_02'): 2.624402,
    (8, 'feature_03'): 2.577300,
    (8, 'feature_04'): 0.110531,
    (8, 'feature_05'): 0.475296,
    (8, 'feature_06'): 1.469630,
    (8, 'feature_07'): 1.600761,
    (8, 'feature_08'): 0.195485,
    (8, 'feature_09'): 15.000000,
    (8, 'feature_10'): 1.000000,
    (8, 'feature_11'): 62.000000,
    (8, 'feature_12'): 0.795884,
    (8, 'feature_13'): 0.677544,
    (8, 'feature_14'): 1.053085,
    (8, 'feature_15'): -0.703753,
    (8, 'feature_16'): -0.648522,
    (8, 'feature_17'): -0.575345,
    (8, 'feature_18'): 2.071006,
    (8, 'feature_19'): -0.615069,
    (8, 'feature_20'): 0.704664,
    (8, 'feature_21'): 0.009882,
    (8, 'feature_22'): 0.303056,
    (8, 'feature_23'): 0.746384,
    (8, 'feature_24'): 0.867018,
    (8, 'feature_25'): 0.624447,
    (8, 'feature_26'): 0.442407,
    (8, 'feature_27'): -0.241958,
    (8, 'feature_28'): -0.485004,
    (8, 'feature_29'): -0.513205,
    (8, 'feature_30'): -0.654379,
    (8, 'feature_31'): 0.009058,
    (8, 'feature_32'): 3.409774,
    (8, 'feature_33'): -0.584268,
    (8, 'feature_34'): 2.689721,
    (8, 'feature_35'): 3.356443,
    (8, 'feature_36'): -1.385308,
    (8, 'feature_37'): 0.067156,
    (8, 'feature_38'): -0.093246,
    (8, 'feature_39'): 0.611184,
    (8, 'feature_40'): 0.076099,
    (8, 'feature_41'): 0.853580,
    (8, 'feature_42'): 0.593495,
    (8, 'feature_43'): -0.268106,
    (8, 'feature_44'): 0.499658,
    (8, 'feature_45'): 0.384322,
    (8, 'feature_46'): 0.633652,
    (8, 'feature_47'): 0.866360,
    (8, 'feature_48'): 0.429342,
    (8, 'feature_49'): 1.320246,
    (8, 'feature_50'): 1.479824,
    (8, 'feature_51'): -0.827326,
    (8, 'feature_52'): 1.811739,
    (8, 'feature_53'): 1.592333,
    (8, 'feature_54'): 0.321968,
    (8, 'feature_55'): 1.741068,
    (8, 'feature_56'): 0.254053,
    (8, 'feature_57'): 1.228932,
    (8, 'feature_58'): 1.481554,
    (8, 'feature_59'): 0.546374,
    (8, 'feature_60'): 0.952322,
    (8, 'feature_61'): -1.101531,
    (8, 'feature_62'): -0.581036,
    (8, 'feature_63'): -0.377870,
    (8, 'feature_64'): -0.407413,
    (8, 'feature_65'): 2.551333,
    (8, 'feature_66'): -1.174801,
    (8, 'feature_67'): 0.323678,
    (8, 'feature_68'): 0.775966,
    (8, 'feature_69'): 0.365464,
    (8, 'feature_70'): 0.789285,
    (8, 'feature_71'): 1.061273,
    (8, 'feature_72'): 1.841206,
    (8, 'feature_73'): 0.366332,
    (8, 'feature_74'): 0.324167,
    (8, 'feature_75'): 0.215918,
    (8, 'feature_76'): 0.262378,
    (8, 'feature_77'): 0.303425,
    (8, 'feature_78'): 0.228453,
    (9, 'feature_00'): 2.644628,
    (9, 'feature_01'): -1.113979,
    (9, 'feature_02'): 2.277499,
    (9, 'feature_03'): 2.575479,
    (9, 'feature_04'): 0.020705,
    (9, 'feature_05'): 0.897692,
    (9, 'feature_06'): 1.484113,
    (9, 'feature_07'): 1.850126,
    (9, 'feature_08'): 0.243260,
    (9, 'feature_09'): 11.000000,
    (9, 'feature_10'): 7.000000,
    (9, 'feature_11'): 76.000000,
    (9, 'feature_12'): 0.266432,
    (9, 'feature_13'): 0.205685,
    (9, 'feature_14'): 0.117142,
    (9, 'feature_15'): -0.661496,
    (9, 'feature_16'): -0.570953,
    (9, 'feature_17'): -0.597975,
    (9, 'feature_18'): -0.898686,
    (9, 'feature_19'): 1.370277,
    (9, 'feature_20'): -0.227114,
    (9, 'feature_21'): -0.298521,
    (9, 'feature_22'): -0.363460,
    (9, 'feature_23'): 0.159058,
    (9, 'feature_24'): 0.348198,
    (9, 'feature_25'): -0.122699,
    (9, 'feature_26'): 3.244771,
    (9, 'feature_27'): 1.491218,
    (9, 'feature_28'): -0.138748,
    (9, 'feature_29'): -0.576149,
    (9, 'feature_30'): -0.462923,
    (9, 'feature_31'): -0.269672,
    (9, 'feature_32'): 2.940819,
    (9, 'feature_33'): -0.432930,
    (9, 'feature_34'): 2.945795,
    (9, 'feature_35'): 2.582282,
    (9, 'feature_36'): -1.297836,
    (9, 'feature_37'): -0.547117,
    (9, 'feature_38'): -0.693353,
    (9, 'feature_39'): -0.542244,
    (9, 'feature_40'): -0.256459,
    (9, 'feature_41'): -0.453220,
    (9, 'feature_42'): 0.359113,
    (9, 'feature_43'): -0.372448,
    (9, 'feature_44'): -0.893519,
    (9, 'feature_45'): -0.622823,
    (9, 'feature_46'): 0.299125,
    (9, 'feature_47'): -0.196535,
    (9, 'feature_48'): -0.588170,
    (9, 'feature_49'): -0.233236,
    (9, 'feature_50'): 0.264139,
    (9, 'feature_51'): -0.321418,
    (9, 'feature_52'): -0.894433,
    (9, 'feature_53'): 1.822091,
    (9, 'feature_54'): 0.768283,
    (9, 'feature_55'): 1.832038,
    (9, 'feature_56'): 0.053750,
    (9, 'feature_57'): -0.013541,
    (9, 'feature_58'): 0.099502,
    (9, 'feature_59'): 0.128364,
    (9, 'feature_60'): 0.347156,
    (9, 'feature_61'): -1.101531,
    (9, 'feature_62'): -0.302044,
    (9, 'feature_63'): -0.210311,
    (9, 'feature_64'): -0.307552,
    (9, 'feature_65'): -0.715832,
    (9, 'feature_66'): 1.817968,
    (9, 'feature_67'): 0.077696,
    (9, 'feature_68'): -0.265932,
    (9, 'feature_69'): -0.297401,
    (9, 'feature_70'): 0.445565,
    (9, 'feature_71'): 0.796879,
    (9, 'feature_72'): 0.585805,
    (9, 'feature_73'): -0.295508,
    (9, 'feature_74'): -0.374614,
    (9, 'feature_75'): -0.188613,
    (9, 'feature_76'): -0.262701,
    (9, 'feature_77'): -0.154700,
    (9, 'feature_78'): -0.239308,
    (10, 'feature_00'): 2.598562,
    (10, 'feature_01'): -1.160995,
    (10, 'feature_02'): 2.762096,
    (10, 'feature_03'): 2.628393,
    (10, 'feature_04'): 0.577654,
    (10, 'feature_05'): 1.050401,
    (10, 'feature_06'): 1.191148,
    (10, 'feature_07'): 2.417670,
    (10, 'feature_08'): 0.222321,
    (10, 'feature_09'): 42.000000,
    (10, 'feature_10'): 5.000000,
    (10, 'feature_11'): 150.000000,
    (10, 'feature_12'): 0.519104,
    (10, 'feature_13'): 0.291731,
    (10, 'feature_14'): 0.035516,
    (10, 'feature_15'): -0.716392,
    (10, 'feature_16'): -0.459941,
    (10, 'feature_17'): -0.631273,
    (10, 'feature_18'): 2.064946,
    (10, 'feature_19'): 1.590741,
    (10, 'feature_20'): 0.280008,
    (10, 'feature_21'): -0.225969,
    (10, 'feature_22'): 0.563274,
    (10, 'feature_23'): 0.659633,
    (10, 'feature_24'): 0.314625,
    (10, 'feature_25'): -0.133646,
    (10, 'feature_26'): 1.371243,
    (10, 'feature_27'): 1.257332,
    (10, 'feature_28'): 0.088097,
    (10, 'feature_29'): -0.644414,
    (10, 'feature_30'): -1.021897,
    (10, 'feature_31'): -0.217222,
    (10, 'feature_32'): 3.695179,
    (10, 'feature_33'): -0.448288,
    (10, 'feature_34'): 3.185451,
    (10, 'feature_35'): 3.093265,
    (10, 'feature_36'): -0.699697,
    (10, 'feature_37'): -0.455727,
    (10, 'feature_38'): -0.571036,
    (10, 'feature_39'): 1.577287,
    (10, 'feature_40'): 0.083801,
    (10, 'feature_41'): 1.393756,
    (10, 'feature_42'): 0.347821,
    (10, 'feature_43'): -0.489232,
    (10, 'feature_44'): 0.197039,
    (10, 'feature_45'): 0.586623,
    (10, 'feature_46'): 0.517844,
    (10, 'feature_47'): 1.148765,
    (10, 'feature_48'): 1.037071,
    (10, 'feature_49'): 1.415310,
    (10, 'feature_50'): 1.329617,
    (10, 'feature_51'): 0.555143,
    (10, 'feature_52'): 1.077234,
    (10, 'feature_53'): 1.238546,
    (10, 'feature_54'): -0.454381,
    (10, 'feature_55'): 0.292146,
    (10, 'feature_56'): 0.193967,
    (10, 'feature_57'): 1.720878,
    (10, 'feature_58'): 1.665463,
    (10, 'feature_59'): 1.234199,
    (10, 'feature_60'): 1.414683,
    (10, 'feature_61'): -1.101531,
    (10, 'feature_62'): -0.404380,
    (10, 'feature_63'): -0.326583,
    (10, 'feature_64'): -0.467636,
    (10, 'feature_65'): 2.424919,
    (10, 'feature_66'): -0.509905,
    (10, 'feature_67'): 0.511438,
    (10, 'feature_68'): -0.031758,
    (10, 'feature_69'): -0.141241,
    (10, 'feature_70'): 0.610248,
    (10, 'feature_71'): 0.460214,
    (10, 'feature_72'): 0.311728,
    (10, 'feature_73'): -0.060785,
    (10, 'feature_74'): 0.008300,
    (10, 'feature_75'): 1.303158,
    (10, 'feature_76'): 0.841004,
    (10, 'feature_77'): 0.259496,
    (10, 'feature_78'): 0.293028,
    (11, 'feature_00'): 2.557203,
    (11, 'feature_01'): -1.061826,
    (11, 'feature_02'): 2.127367,
    (11, 'feature_03'): 2.565700,
    (11, 'feature_04'): 0.182079,
    (11, 'feature_05'): 0.891533,
    (11, 'feature_06'): 1.759385,
    (11, 'feature_07'): 2.253046,
    (11, 'feature_08'): 0.338756,
    (11, 'feature_09'): 46.000000,
    (11, 'feature_10'): 3.000000,
    (11, 'feature_11'): 261.000000,
    (11, 'feature_12'): 1.265264,
    (11, 'feature_13'): 0.678300,
    (11, 'feature_14'): 0.733056,
    (11, 'feature_15'): -0.954659,
    (11, 'feature_16'): -0.525153,
    (11, 'feature_17'): -0.901868,
    (11, 'feature_18'): 2.780208,
    (11, 'feature_19'): 1.534860,
    (11, 'feature_20'): -0.554231,
    (11, 'feature_21'): -0.175924,
    (11, 'feature_22'): 0.213529,
    (11, 'feature_23'): 0.369315,
    (11, 'feature_24'): -0.620379,
    (11, 'feature_25'): -1.444003,
    (11, 'feature_26'): 0.018664,
    (11, 'feature_27'): 0.251433,
    (11, 'feature_28'): 0.121319,
    (11, 'feature_29'): -0.601176,
    (11, 'feature_30'): -0.855676,
    (11, 'feature_31'): -0.198678,
    (11, 'feature_32'): 2.771396,
    (11, 'feature_33'): -0.356122,
    (11, 'feature_34'): 3.420637,
    (11, 'feature_35'): 3.377889,
    (11, 'feature_36'): -0.559840,
    (11, 'feature_37'): -0.108718,
    (11, 'feature_38'): -0.174752,
    (11, 'feature_39'): 0.790170,
    (11, 'feature_40'): -0.391075,
    (11, 'feature_41'): 0.588243,
    (11, 'feature_42'): 0.932431,
    (11, 'feature_43'): -0.458814,
    (11, 'feature_44'): 0.383988,
    (11, 'feature_45'): 0.090712,
    (11, 'feature_46'): -0.156561,
    (11, 'feature_47'): 0.316766,
    (11, 'feature_48'): 0.075364,
    (11, 'feature_49'): 0.726153,
    (11, 'feature_50'): 0.988728,
    (11, 'feature_51'): 0.275304,
    (11, 'feature_52'): -0.275015,
    (11, 'feature_53'): 1.584332,
    (11, 'feature_54'): 0.004580,
    (11, 'feature_55'): 0.301711,
    (11, 'feature_56'): -0.224861,
    (11, 'feature_57'): 0.069382,
    (11, 'feature_58'): 0.612345,
    (11, 'feature_59'): 0.107333,
    (11, 'feature_60'): 1.298290,
    (11, 'feature_61'): -1.101531,
    (11, 'feature_62'): -0.394523,
    (11, 'feature_63'): -0.393180,
    (11, 'feature_64'): -0.350247,
    (11, 'feature_65'): 2.398663,
    (11, 'feature_66'): 1.430195,
    (11, 'feature_67'): 1.157438,
    (11, 'feature_68'): 0.438987,
    (11, 'feature_69'): 0.274677,
    (11, 'feature_70'): 1.416519,
    (11, 'feature_71'): 0.626940,
    (11, 'feature_72'): 1.158743,
    (11, 'feature_73'): -0.350792,
    (11, 'feature_74'): -0.241679,
    (11, 'feature_75'): -0.229022,
    (11, 'feature_76'): -0.274400,
    (11, 'feature_77'): -0.285774,
    (11, 'feature_78'): -0.206792,
    (12, 'feature_00'): 1.952569,
    (12, 'feature_01'): -1.110201,
    (12, 'feature_02'): 1.939148,
    (12, 'feature_03'): 2.506562,
    (12, 'feature_04'): 0.744245,
    (12, 'feature_05'): 0.386946,
    (12, 'feature_06'): 1.270716,
    (12, 'feature_07'): 1.299021,
    (12, 'feature_08'): 0.263218,
    (12, 'feature_09'): 70.000000,
    (12, 'feature_10'): 7.000000,
    (12, 'feature_11'): 410.000000,
    (12, 'feature_12'): 0.406752,
    (12, 'feature_13'): 3.929572,
    (12, 'feature_14'): 1.129731,
    (12, 'feature_15'): -0.796767,
    (12, 'feature_16'): -0.488023,
    (12, 'feature_17'): -0.742959,
    (12, 'feature_18'): -1.057587,
    (12, 'feature_19'): 0.064147,
    (12, 'feature_20'): -0.395036,
    (12, 'feature_21'): 0.513603,
    (12, 'feature_22'): 1.321559,
    (12, 'feature_23'): 1.297216,
    (12, 'feature_24'): 1.232881,
    (12, 'feature_25'): 1.203859,
    (12, 'feature_26'): 0.572971,
    (12, 'feature_27'): -0.530059,
    (12, 'feature_28'): -0.782427,
    (12, 'feature_29'): -0.467606,
    (12, 'feature_30'): -0.646224,
    (12, 'feature_31'): 0.598179,
    (12, 'feature_32'): 2.966184,
    (12, 'feature_33'): -1.260832,
    (12, 'feature_34'): 3.779864,
    (12, 'feature_35'): 3.073408,
    (12, 'feature_36'): -0.853924,
    (12, 'feature_37'): -0.210959,
    (12, 'feature_38'): -0.386617,
    (12, 'feature_39'): 0.790938,
    (12, 'feature_40'): 0.722761,
    (12, 'feature_41'): 0.953989,
    (12, 'feature_42'): -0.121679,
    (12, 'feature_43'): -1.548448,
    (12, 'feature_44'): -0.710001,
    (12, 'feature_45'): -0.084562,
    (12, 'feature_46'): -0.448785,
    (12, 'feature_47'): 0.077220,
    (12, 'feature_48'): 0.163891,
    (12, 'feature_49'): 0.853235,
    (12, 'feature_50'): 1.810012,
    (12, 'feature_51'): 0.490674,
    (12, 'feature_52'): 1.709234,
    (12, 'feature_53'): -0.754559,
    (12, 'feature_54'): -2.553131,
    (12, 'feature_55'): -1.563670,
    (12, 'feature_56'): -0.207414,
    (12, 'feature_57'): -0.915344,
    (12, 'feature_58'): 0.386669,
    (12, 'feature_59'): 1.302667,
    (12, 'feature_60'): 2.213575,
    (12, 'feature_61'): -1.101531,
    (12, 'feature_62'): -0.545590,
    (12, 'feature_63'): -0.320662,
    (12, 'feature_64'): -0.462639,
    (12, 'feature_65'): -0.551901,
    (12, 'feature_66'): 0.439948,
    (12, 'feature_67'): 0.365354,
    (12, 'feature_68'): 0.307506,
    (12, 'feature_69'): 0.420086,
    (12, 'feature_70'): 0.386310,
    (12, 'feature_71'): 6.259795,
    (12, 'feature_72'): 1.885505,
    (12, 'feature_73'): -0.432059,
    (12, 'feature_74'): -0.334890,
    (12, 'feature_75'): 0.227920,
    (12, 'feature_76'): 0.148177,
    (12, 'feature_77'): -0.160401,
    (12, 'feature_78'): -0.191864,
    (13, 'feature_00'): 2.664488,
    (13, 'feature_01'): -1.532176,
    (13, 'feature_02'): 2.357599,
    (13, 'feature_03'): 2.232284,
    (13, 'feature_04'): 0.498488,
    (13, 'feature_05'): 0.746604,
    (13, 'feature_06'): 1.038940,
    (13, 'feature_07'): 2.562644,
    (13, 'feature_08'): 0.340430,
    (13, 'feature_09'): 50.000000,
    (13, 'feature_10'): 1.000000,
    (13, 'feature_11'): 522.000000,
    (13, 'feature_12'): 2.313901,
    (13, 'feature_13'): 1.898254,
    (13, 'feature_14'): 0.704617,
    (13, 'feature_15'): -0.766220,
    (13, 'feature_16'): -0.455401,
    (13, 'feature_17'): -0.423305,
    (13, 'feature_18'): 2.599999,
    (13, 'feature_19'): 1.511365,
    (13, 'feature_20'): 0.188037,
    (13, 'feature_21'): -0.003101,
    (13, 'feature_22'): 0.142380,
    (13, 'feature_23'): 0.550107,
    (13, 'feature_24'): 0.158455,
    (13, 'feature_25'): -0.436337,
    (13, 'feature_26'): 3.619649,
    (13, 'feature_27'): -0.383218,
    (13, 'feature_28'): -1.434520,
    (13, 'feature_29'): -0.579597,
    (13, 'feature_30'): -0.749147,
    (13, 'feature_31'): -0.002427,
    (13, 'feature_32'): 2.773225,
    (13, 'feature_33'): -0.662348,
    (13, 'feature_34'): 3.101244,
    (13, 'feature_35'): 2.835237,
    (13, 'feature_36'): -0.842369,
    (13, 'feature_37'): 0.400331,
    (13, 'feature_38'): 0.108143,
    (13, 'feature_39'): 1.911082,
    (13, 'feature_40'): 0.160848,
    (13, 'feature_41'): 0.305224,
    (13, 'feature_42'): 0.272355,
    (13, 'feature_43'): -0.285889,
    (13, 'feature_44'): -0.275729,
    (13, 'feature_45'): -0.370835,
    (13, 'feature_46'): -0.936010,
    (13, 'feature_47'): -0.739831,
    (13, 'feature_48'): 0.464731,
    (13, 'feature_49'): 0.580374,
    (13, 'feature_50'): 1.818290,
    (13, 'feature_51'): 1.254338,
    (13, 'feature_52'): 0.280416,
    (13, 'feature_53'): 2.117959,
    (13, 'feature_54'): -0.115561,
    (13, 'feature_55'): 1.364943,
    (13, 'feature_56'): 0.353682,
    (13, 'feature_57'): -1.153529,
    (13, 'feature_58'): -0.333410,
    (13, 'feature_59'): 1.310358,
    (13, 'feature_60'): 1.438193,
    (13, 'feature_61'): -1.101531,
    (13, 'feature_62'): -0.519599,
    (13, 'feature_63'): -0.345891,
    (13, 'feature_64'): -0.488490,
    (13, 'feature_65'): 2.807424,
    (13, 'feature_66'): 1.434260,
    (13, 'feature_67'): 2.335643,
    (13, 'feature_68'): 1.085945,
    (13, 'feature_69'): 0.879603,
    (13, 'feature_70'): 1.984499,
    (13, 'feature_71'): 1.062641,
    (13, 'feature_72'): 1.321187,
    (13, 'feature_73'): -0.228078,
    (13, 'feature_74'): -0.367905,
    (13, 'feature_75'): -0.159154,
    (13, 'feature_76'): -0.134687,
    (13, 'feature_77'): -0.142200,
    (13, 'feature_78'): -0.191635,
    (14, 'feature_00'): 2.609008,
    (14, 'feature_01'): -0.879530,
    (14, 'feature_02'): 1.989910,
    (14, 'feature_03'): 1.823826,
    (14, 'feature_04'): -0.132923,
    (14, 'feature_05'): 0.854801,
    (14, 'feature_06'): 1.740368,
    (14, 'feature_07'): 1.258237,
    (14, 'feature_08'): 0.254392,
    (14, 'feature_09'): 44.000000,
    (14, 'feature_10'): 3.000000,
    (14, 'feature_11'): 16.000000,
    (14, 'feature_12'): 0.410492,
    (14, 'feature_13'): 0.246258,
    (14, 'feature_14'): 0.249036,
    (14, 'feature_15'): -0.645515,
    (14, 'feature_16'): -0.511915,
    (14, 'feature_17'): -0.671983,
    (14, 'feature_18'): 2.115987,
    (14, 'feature_19'): -0.877937,
    (14, 'feature_20'): 0.119601,
    (14, 'feature_21'): -0.213447,
    (14, 'feature_22'): -0.237826,
    (14, 'feature_23'): 0.095793,
    (14, 'feature_24'): -0.296660,
    (14, 'feature_25'): -0.854181,
    (14, 'feature_26'): -0.697137,
    (14, 'feature_27'): -0.150428,
    (14, 'feature_28'): -0.145329,
    (14, 'feature_29'): -0.543692,
    (14, 'feature_30'): -0.799246,
    (14, 'feature_31'): -0.252426,
    (14, 'feature_32'): 2.940065,
    (14, 'feature_33'): -1.231904,
    (14, 'feature_34'): 3.179842,
    (14, 'feature_35'): 3.489544,
    (14, 'feature_36'): -2.124364,
    (14, 'feature_37'): 0.161757,
    (14, 'feature_38'): 0.028187,
    (14, 'feature_39'): 1.377599,
    (14, 'feature_40'): -0.038178,
    (14, 'feature_41'): 1.241772,
    (14, 'feature_42'): 1.124562,
    (14, 'feature_43'): -0.334527,
    (14, 'feature_44'): -0.035669,
    (14, 'feature_45'): 0.015255,
    (14, 'feature_46'): -1.215203,
    (14, 'feature_47'): 0.631621,
    (14, 'feature_48'): 0.237648,
    (14, 'feature_49'): 1.618853,
    (14, 'feature_50'): 1.769095,
    (14, 'feature_51'): 0.838718,
    (14, 'feature_52'): 1.491907,
    (14, 'feature_53'): 2.285187,
    (14, 'feature_54'): 0.834260,
    (14, 'feature_55'): 1.616372,
    (14, 'feature_56'): 0.593562,
    (14, 'feature_57'): 0.558040,
    (14, 'feature_58'): 1.714653,
    (14, 'feature_59'): 0.716379,
    (14, 'feature_60'): 1.625788,
    (14, 'feature_61'): -1.101531,
    (14, 'feature_62'): -0.271118,
    (14, 'feature_63'): -0.308695,
    (14, 'feature_64'): -0.258927,
    (14, 'feature_65'): 2.046615,
    (14, 'feature_66'): -1.080599,
    (14, 'feature_67'): 0.353432,
    (14, 'feature_68'): -0.160770,
    (14, 'feature_69'): -0.085363,
    (14, 'feature_70'): 0.555833,
    (14, 'feature_71'): 0.702702,
    (14, 'feature_72'): 0.653302,
    (14, 'feature_73'): -0.126576,
    (14, 'feature_74'): -0.072147,
    (14, 'feature_75'): 0.150474,
    (14, 'feature_76'): 0.186843,
    (14, 'feature_77'): 0.022553,
    (14, 'feature_78'): 0.014052,
    (15, 'feature_00'): 2.071874,
    (15, 'feature_01'): -1.127766,
    (15, 'feature_02'): 2.736025,
    (15, 'feature_03'): 2.369568,
    (15, 'feature_04'): 0.125078,
    (15, 'feature_05'): 0.729738,
    (15, 'feature_06'): 1.631535,
    (15, 'feature_07'): 2.260375,
    (15, 'feature_08'): 0.343890,
    (15, 'feature_09'): 9.000000,
    (15, 'feature_10'): 3.000000,
    (15, 'feature_11'): 25.000000,
    (15, 'feature_12'): 0.737310,
    (15, 'feature_13'): 0.290446,
    (15, 'feature_14'): 0.228912,
    (15, 'feature_15'): -0.810100,
    (15, 'feature_16'): -0.421244,
    (15, 'feature_17'): -0.749110,
    (15, 'feature_18'): 0.619860,
    (15, 'feature_19'): 1.313866,
    (15, 'feature_20'): -0.441937,
    (15, 'feature_21'): -0.158947,
    (15, 'feature_22'): -0.252724,
    (15, 'feature_23'): 0.038192,
    (15, 'feature_24'): -0.136334,
    (15, 'feature_25'): -1.037343,
    (15, 'feature_26'): -1.095486,
    (15, 'feature_27'): -0.086359,
    (15, 'feature_28'): 0.003706,
    (15, 'feature_29'): -0.764941,
    (15, 'feature_30'): -0.734403,
    (15, 'feature_31'): -0.253705,
    (15, 'feature_32'): 3.124130,
    (15, 'feature_33'): -0.771925,
    (15, 'feature_34'): 2.843634,
    (15, 'feature_35'): 2.687504,
    (15, 'feature_36'): 1.360286,
    (15, 'feature_37'): 0.152335,
    (15, 'feature_38'): 0.015398,
    (15, 'feature_39'): 1.813750,
    (15, 'feature_40'): -0.146147,
    (15, 'feature_41'): 0.997223,
    (15, 'feature_42'): 0.620095,
    (15, 'feature_43'): 0.082732,
    (15, 'feature_44'): 0.776374,
    (15, 'feature_45'): 0.552601,
    (15, 'feature_46'): 0.216579,
    (15, 'feature_47'): 0.258927,
    (15, 'feature_48'): 0.715627,
    (15, 'feature_49'): 0.912468,
    (15, 'feature_50'): 0.309951,
    (15, 'feature_51'): 0.038762,
    (15, 'feature_52'): 0.696644,
    (15, 'feature_53'): 1.382980,
    (15, 'feature_54'): -0.507114,
    (15, 'feature_55'): 0.382910,
    (15, 'feature_56'): 0.348553,
    (15, 'feature_57'): 0.740945,
    (15, 'feature_58'): 0.553801,
    (15, 'feature_59'): 0.007453,
    (15, 'feature_60'): 0.571862,
    (15, 'feature_61'): -1.101531,
    (15, 'feature_62'): -0.299673,
    (15, 'feature_63'): -0.297143,
    (15, 'feature_64'): -0.359751,
    (15, 'feature_65'): 0.741609,
    (15, 'feature_66'): -1.511563,
    (15, 'feature_67'): 0.476926,
    (15, 'feature_68'): 0.230976,
    (15, 'feature_69'): 0.011984,
    (15, 'feature_70'): 1.081988,
    (15, 'feature_71'): 0.282066,
    (15, 'feature_72'): 0.518758,
    (15, 'feature_73'): 0.413760,
    (15, 'feature_74'): 0.299256,
    (15, 'feature_75'): 0.917401,
    (15, 'feature_76'): 1.037147,
    (15, 'feature_77'): 0.381928,
    (15, 'feature_78'): 0.321158,
    (16, 'feature_00'): 2.584535,
    (16, 'feature_01'): -0.768900,
    (16, 'feature_02'): 2.113786,
    (16, 'feature_03'): 2.170725,
    (16, 'feature_04'): 0.622436,
    (16, 'feature_05'): 0.559597,
    (16, 'feature_06'): 1.018920,
    (16, 'feature_07'): 1.376493,
    (16, 'feature_08'): 0.225189,
    (16, 'feature_09'): 11.000000,
    (16, 'feature_10'): 7.000000,
    (16, 'feature_11'): 76.000000,
    (16, 'feature_12'): 0.379036,
    (16, 'feature_13'): 0.529474,
    (16, 'feature_14'): 0.003028,
    (16, 'feature_15'): -0.610329,
    (16, 'feature_16'): -0.505230,
    (16, 'feature_17'): -0.661778,
    (16, 'feature_18'): 2.173470,
    (16, 'feature_19'): 0.175416,
    (16, 'feature_20'): 1.655843,
    (16, 'feature_21'): -0.061361,
    (16, 'feature_22'): 1.784431,
    (16, 'feature_23'): 1.331869,
    (16, 'feature_24'): 0.938268,
    (16, 'feature_25'): 0.536201,
    (16, 'feature_26'): -0.657209,
    (16, 'feature_27'): 0.584148,
    (16, 'feature_28'): 1.005504,
    (16, 'feature_29'): -0.594820,
    (16, 'feature_30'): -0.942611,
    (16, 'feature_31'): -0.054635,
    (16, 'feature_32'): 3.128937,
    (16, 'feature_33'): -0.531331,
    (16, 'feature_34'): 3.219064,
    (16, 'feature_35'): 3.433126,
    (16, 'feature_36'): 0.687542,
    (16, 'feature_37'): 0.021301,
    (16, 'feature_38'): -0.109133,
    (16, 'feature_39'): 0.425991,
    (16, 'feature_40'): 0.684570,
    (16, 'feature_41'): 0.452680,
    (16, 'feature_42'): 1.958628,
    (16, 'feature_43'): 0.097508,
    (16, 'feature_44'): 0.570226,
    (16, 'feature_45'): -0.557373,
    (16, 'feature_46'): -0.850096,
    (16, 'feature_47'): 0.038230,
    (16, 'feature_48'): 1.350792,
    (16, 'feature_49'): 0.670567,
    (16, 'feature_50'): 0.199469,
    (16, 'feature_51'): 0.865753,
    (16, 'feature_52'): 0.251012,
    (16, 'feature_53'): 2.445872,
    (16, 'feature_54'): 0.351834,
    (16, 'feature_55'): 1.720947,
    (16, 'feature_56'): -1.081315,
    (16, 'feature_57'): -0.811772,
    (16, 'feature_58'): 0.032490,
    (16, 'feature_59'): 1.452237,
    (16, 'feature_60'): 0.861393,
    (16, 'feature_61'): -1.101531,
    (16, 'feature_62'): -0.480174,
    (16, 'feature_63'): -0.469069,
    (16, 'feature_64'): -0.422925,
    (16, 'feature_65'): 2.310571,
    (16, 'feature_66'): 0.299895,
    (16, 'feature_67'): 0.255005,
    (16, 'feature_68'): -0.053193,
    (16, 'feature_69'): -0.295578,
    (16, 'feature_70'): 0.382134,
    (16, 'feature_71'): 0.817339,
    (16, 'feature_72'): 0.417095,
    (16, 'feature_73'): -0.277198,
    (16, 'feature_74'): -0.298969,
    (16, 'feature_75'): -0.237268,
    (16, 'feature_76'): -0.174465,
    (16, 'feature_77'): -0.212034,
    (16, 'feature_78'): -0.298271,
    (17, 'feature_00'): 2.208030,
    (17, 'feature_01'): -0.709342,
    (17, 'feature_02'): 2.497351,
    (17, 'feature_03'): 2.281530,
    (17, 'feature_04'): 0.832571,
    (17, 'feature_05'): 0.671607,
    (17, 'feature_06'): 2.094076,
    (17, 'feature_07'): 2.779384,
    (17, 'feature_08'): 0.256405,
    (17, 'feature_09'): 64.000000,
    (17, 'feature_10'): 6.000000,
    (17, 'feature_11'): 376.000000,
    (17, 'feature_12'): 0.929008,
    (17, 'feature_13'): 0.678354,
    (17, 'feature_14'): 0.960326,
    (17, 'feature_15'): -0.623416,
    (17, 'feature_16'): -0.588481,
    (17, 'feature_17'): -0.333105,
    (17, 'feature_18'): -0.112061,
    (17, 'feature_19'): 2.062315,
    (17, 'feature_20'): -0.239813,
    (17, 'feature_21'): -0.191956,
    (17, 'feature_22'): 1.212122,
    (17, 'feature_23'): 1.575729,
    (17, 'feature_24'): -0.368070,
    (17, 'feature_25'): -0.926467,
    (17, 'feature_26'): 0.331012,
    (17, 'feature_27'): 1.235588,
    (17, 'feature_28'): 1.522907,
    (17, 'feature_29'): -0.615122,
    (17, 'feature_30'): -0.534068,
    (17, 'feature_31'): -0.165182,
    (17, 'feature_32'): 3.079577,
    (17, 'feature_33'): -1.618836,
    (17, 'feature_34'): 3.033669,
    (17, 'feature_35'): 3.206255,
    (17, 'feature_36'): -1.513128,
    (17, 'feature_37'): -0.248568,
    (17, 'feature_38'): -0.128724,
    (17, 'feature_39'): 1.617820,
    (17, 'feature_40'): 0.628400,
    (17, 'feature_41'): 0.862016,
    (17, 'feature_42'): 0.875266,
    (17, 'feature_43'): -0.072540,
    (17, 'feature_44'): 0.124076,
    (17, 'feature_45'): 0.885494,
    (17, 'feature_46'): -0.364029,
    (17, 'feature_47'): 0.644940,
    (17, 'feature_48'): 0.524538,
    (17, 'feature_49'): 1.228554,
    (17, 'feature_50'): 1.678863,
    (17, 'feature_51'): -0.756849,
    (17, 'feature_52'): 0.461680,
    (17, 'feature_53'): 0.632411,
    (17, 'feature_54'): -1.052518,
    (17, 'feature_55'): -0.068747,
    (17, 'feature_56'): 1.103811,
    (17, 'feature_57'): 0.486236,
    (17, 'feature_58'): 1.180702,
    (17, 'feature_59'): 0.231019,
    (17, 'feature_60'): 0.681097,
    (17, 'feature_61'): -1.101531,
    (17, 'feature_62'): -0.317082,
    (17, 'feature_63'): -0.316534,
    (17, 'feature_64'): -0.406886,
    (17, 'feature_65'): -0.508665,
    (17, 'feature_66'): 1.471353,
    (17, 'feature_67'): 1.121421,
    (17, 'feature_68'): 0.677412,
    (17, 'feature_69'): 0.610699,
    (17, 'feature_70'): 0.789447,
    (17, 'feature_71'): 0.369597,
    (17, 'feature_72'): 0.742339,
    (17, 'feature_73'): -0.365186,
    (17, 'feature_74'): -0.225510,
    (17, 'feature_75'): -0.271926,
    (17, 'feature_76'): -0.227629,
    (17, 'feature_77'): -0.199733,
    (17, 'feature_78'): -0.167553,
    (18, 'feature_00'): 1.932134,
    (18, 'feature_01'): -0.762696,
    (18, 'feature_02'): 2.503707,
    (18, 'feature_03'): 2.525652,
    (18, 'feature_04'): 0.273878,
    (18, 'feature_05'): 0.643094,
    (18, 'feature_06'): 0.774099,
    (18, 'feature_07'): 1.823852,
    (18, 'feature_08'): 0.210567,
    (18, 'feature_09'): 15.000000,
    (18, 'feature_10'): 1.000000,
    (18, 'feature_11'): 66.000000,
    (18, 'feature_12'): 0.311808,
    (18, 'feature_13'): 0.745650,
    (18, 'feature_14'): 0.470787,
    (18, 'feature_15'): -0.325203,
    (18, 'feature_16'): -0.745984,
    (18, 'feature_17'): -0.523376,
    (18, 'feature_18'): -0.109944,
    (18, 'feature_19'): 1.844099,
    (18, 'feature_20'): -0.359065,
    (18, 'feature_21'): -0.113343,
    (18, 'feature_22'): 0.011076,
    (18, 'feature_23'): -0.081742,
    (18, 'feature_24'): 0.974965,
    (18, 'feature_25'): 0.267692,
    (18, 'feature_26'): 0.220525,
    (18, 'feature_27'): -0.562124,
    (18, 'feature_28'): -0.577864,
    (18, 'feature_29'): -0.657656,
    (18, 'feature_30'): -0.650069,
    (18, 'feature_31'): -0.071138,
    (18, 'feature_32'): 2.859055,
    (18, 'feature_33'): -1.807279,
    (18, 'feature_34'): 3.030884,
    (18, 'feature_35'): 3.603027,
    (18, 'feature_36'): 0.947276,
    (18, 'feature_37'): -0.090407,
    (18, 'feature_38'): -0.037630,
    (18, 'feature_39'): 1.331056,
    (18, 'feature_40'): -0.107870,
    (18, 'feature_41'): 0.699056,
    (18, 'feature_42'): 0.428603,
    (18, 'feature_43'): -0.473408,
    (18, 'feature_44'): 0.079432,
    (18, 'feature_45'): 0.219900,
    (18, 'feature_46'): -1.139165,
    (18, 'feature_47'): 0.122939,
    (18, 'feature_48'): -0.071957,
    (18, 'feature_49'): 0.595288,
    (18, 'feature_50'): 1.406533,
    (18, 'feature_51'): -0.345314,
    (18, 'feature_52'): 0.035703,
    (18, 'feature_53'): 1.392462,
    (18, 'feature_54'): 0.140371,
    (18, 'feature_55'): 0.435415,
    (18, 'feature_56'): 0.895936,
    (18, 'feature_57'): 0.847487,
    (18, 'feature_58'): 0.583432,
    (18, 'feature_59'): 0.010144,
    (18, 'feature_60'): 0.651920,
    (18, 'feature_61'): -1.101531,
    (18, 'feature_62'): -0.467438,
    (18, 'feature_63'): -0.220689,
    (18, 'feature_64'): -0.274816,
    (18, 'feature_65'): 2.352730,
    (18, 'feature_66'): 2.213133,
    (18, 'feature_67'): 0.476627,
    (18, 'feature_68'): 0.421219,
    (18, 'feature_69'): 0.118835,
    (18, 'feature_70'): 0.392613,
    (18, 'feature_71'): 0.818812,
    (18, 'feature_72'): 0.646325,
    (18, 'feature_73'): -0.144090,
    (18, 'feature_74'): -0.152558,
    (18, 'feature_75'): -0.124998,
    (18, 'feature_76'): -0.164410,
    (18, 'feature_77'): -0.226233,
    (18, 'feature_78'): -0.202508,
    (19, 'feature_00'): 2.271131,
    (19, 'feature_01'): -0.848758,
    (19, 'feature_02'): 2.839265,
    (19, 'feature_03'): 2.230107,
    (19, 'feature_04'): 0.265316,
    (19, 'feature_05'): 0.927607,
    (19, 'feature_06'): 1.848239,
    (19, 'feature_07'): 1.588164,
    (19, 'feature_08'): 0.296958,
    (19, 'feature_09'): 4.000000,
    (19, 'feature_10'): 3.000000,
    (19, 'feature_11'): 11.000000,
    (19, 'feature_12'): 1.351751,
    (19, 'feature_13'): 1.862462,
    (19, 'feature_14'): 1.045507,
    (19, 'feature_15'): -0.404748,
    (19, 'feature_16'): -0.379189,
    (19, 'feature_17'): -0.669152,
    (19, 'feature_18'): 2.610760,
    (19, 'feature_19'): 1.405005,
    (19, 'feature_20'): 0.021237,
    (19, 'feature_21'): -0.042261,
    (19, 'feature_22'): 1.792063,
    (19, 'feature_23'): 1.740623,
    (19, 'feature_24'): 0.235926,
    (19, 'feature_25'): -0.205855,
    (19, 'feature_26'): 0.554158,
    (19, 'feature_27'): 1.093280,
    (19, 'feature_28'): 1.389925,
    (19, 'feature_29'): -0.506655,
    (19, 'feature_30'): -0.550083,
    (19, 'feature_31'): -0.095118,
    (19, 'feature_32'): 3.325561,
    (19, 'feature_33'): -0.268881,
    (19, 'feature_34'): 3.596057,
    (19, 'feature_35'): 2.780406,
    (19, 'feature_36'): -1.159114,
    (19, 'feature_37'): 0.301250,
    (19, 'feature_38'): 0.105407,
    (19, 'feature_39'): 0.604078,
    (19, 'feature_40'): -0.010756,
    (19, 'feature_41'): -0.357511,
    (19, 'feature_42'): -0.661826,
    (19, 'feature_43'): -0.730417,
    (19, 'feature_44'): -0.882318,
    (19, 'feature_45'): -0.119932,
    (19, 'feature_46'): -0.614513,
    (19, 'feature_47'): -0.860342,
    (19, 'feature_48'): -1.004300,
    (19, 'feature_49'): -0.490038,
    (19, 'feature_50'): 0.693331,
    (19, 'feature_51'): -0.560676,
    (19, 'feature_52'): 0.055568,
    (19, 'feature_53'): -0.520826,
    (19, 'feature_54'): 0.065971,
    (19, 'feature_55'): 0.135725,
    (19, 'feature_56'): -0.473632,
    (19, 'feature_57'): -1.248763,
    (19, 'feature_58'): -1.346516,
    (19, 'feature_59'): -0.596990,
    (19, 'feature_60'): -0.081497,
    (19, 'feature_61'): -1.101531,
    (19, 'feature_62'): -0.464533,
    (19, 'feature_63'): -0.405523,
    (19, 'feature_64'): -0.327713,
    (19, 'feature_65'): 1.891539,
    (19, 'feature_66'): 2.241728,
    (19, 'feature_67'): 1.592768,
    (19, 'feature_68'): 0.556732,
    (19, 'feature_69'): 0.325458,
    (19, 'feature_70'): 1.484756,
    (19, 'feature_71'): 3.467576,
    (19, 'feature_72'): 1.512103,
    (19, 'feature_73'): -0.276808,
    (19, 'feature_74'): -0.208684,
    (19, 'feature_75'): -0.161863,
    (19, 'feature_76'): -0.169449,
    (19, 'feature_77'): -0.174203,
    (19, 'feature_78'): -0.165020,
    (20, 'feature_00'): 2.811718,
    (20, 'feature_01'): -0.309325,
    (20, 'feature_02'): 2.289834,
    (20, 'feature_03'): 2.520646,
    (20, 'feature_04'): 0.147196,
    (20, 'feature_05'): 0.602508,
    (20, 'feature_06'): 1.641045,
    (20, 'feature_07'): 1.343061,
    (20, 'feature_08'): 0.167476,
    (20, 'feature_09'): 30.000000,
    (20, 'feature_10'): 10.000000,
    (20, 'feature_11'): 50.000000,
    (20, 'feature_12'): 1.614965,
    (20, 'feature_13'): 1.073755,
    (20, 'feature_14'): 2.720595,
    (20, 'feature_15'): -0.991578,
    (20, 'feature_16'): -0.479267,
    (20, 'feature_17'): -0.881280,
    (20, 'feature_18'): 2.585636,
    (20, 'feature_19'): -0.675742,
    (20, 'feature_20'): -0.395041,
    (20, 'feature_21'): -0.179616,
    (20, 'feature_22'): -1.100106,
    (20, 'feature_23'): -0.807257,
    (20, 'feature_24'): 0.860350,
    (20, 'feature_25'): 0.500722,
    (20, 'feature_26'): 1.302139,
    (20, 'feature_27'): -0.994641,
    (20, 'feature_28'): -2.328478,
    (20, 'feature_29'): -0.553408,
    (20, 'feature_30'): -1.063754,
    (20, 'feature_31'): -0.217139,
    (20, 'feature_32'): 2.931128,
    (20, 'feature_33'): -0.411284,
    (20, 'feature_34'): 3.121096,
    (20, 'feature_35'): 3.148288,
    (20, 'feature_36'): 0.537076,
    (20, 'feature_37'): 2.949838,
    (20, 'feature_38'): 2.734550,
    (20, 'feature_39'): 0.631988,
    (20, 'feature_40'): 0.254302,
    (20, 'feature_41'): 0.255388,
    (20, 'feature_42'): -0.900837,
    (20, 'feature_43'): -0.268612,
    (20, 'feature_44'): -1.504900,
    (20, 'feature_45'): 0.257317,
    (20, 'feature_46'): -1.901577,
    (20, 'feature_47'): 0.299895,
    (20, 'feature_48'): -0.146047,
    (20, 'feature_49'): -0.326090,
    (20, 'feature_50'): -0.214521,
    (20, 'feature_51'): -0.466102,
    (20, 'feature_52'): -0.065103,
    (20, 'feature_53'): -0.902761,
    (20, 'feature_54'): 0.573990,
    (20, 'feature_55'): -1.620528,
    (20, 'feature_56'): 0.166226,
    (20, 'feature_57'): -2.060915,
    (20, 'feature_58'): 0.656331,
    (20, 'feature_59'): 0.285737,
    (20, 'feature_60'): -0.481740,
    (20, 'feature_61'): -1.101531,
    (20, 'feature_62'): -0.521766,
    (20, 'feature_63'): -0.288180,
    (20, 'feature_64'): -0.327497,
    (20, 'feature_65'): 2.102297,
    (20, 'feature_66'): -0.759297,
    (20, 'feature_67'): 1.521861,
    (20, 'feature_68'): 0.565366,
    (20, 'feature_69'): 1.000769,
    (20, 'feature_70'): 1.868358,
    (20, 'feature_71'): 1.137155,
    (20, 'feature_72'): 1.723911,
    (20, 'feature_73'): 0.336567,
    (20, 'feature_74'): 0.024787,
    (20, 'feature_75'): 0.130278,
    (20, 'feature_76'): -0.042252,
    (20, 'feature_77'): 0.272658,
    (20, 'feature_78'): 0.003435,
    (21, 'feature_00'): 2.583012,
    (21, 'feature_01'): -0.637597,
    (21, 'feature_02'): 2.571924,
    (21, 'feature_03'): 2.306421,
    (21, 'feature_04'): 0.114189,
    (21, 'feature_05'): 0.733313,
    (21, 'feature_06'): 1.458878,
    (21, 'feature_07'): 1.970754,
    (21, 'feature_08'): 0.326351,
    (21, 'feature_09'): 34.000000,
    (21, 'feature_10'): 4.000000,
    (21, 'feature_11'): 214.000000,
    (21, 'feature_12'): 1.142877,
    (21, 'feature_13'): 0.211828,
    (21, 'feature_14'): 0.886781,
    (21, 'feature_15'): -0.705158,
    (21, 'feature_16'): -0.569825,
    (21, 'feature_17'): -0.563777,
    (21, 'feature_18'): 3.072022,
    (21, 'feature_19'): 1.032438,
    (21, 'feature_20'): -0.481073,
    (21, 'feature_21'): -0.055942,
    (21, 'feature_22'): -0.462917,
    (21, 'feature_23'): -0.016532,
    (21, 'feature_24'): 0.453578,
    (21, 'feature_25'): 0.031210,
    (21, 'feature_26'): -0.580558,
    (21, 'feature_27'): -1.221874,
    (21, 'feature_28'): -0.996345,
    (21, 'feature_29'): -0.616744,
    (21, 'feature_30'): -0.436035,
    (21, 'feature_31'): -0.035300,
    (21, 'feature_32'): 3.126277,
    (21, 'feature_33'): -0.903778,
    (21, 'feature_34'): 3.311344,
    (21, 'feature_35'): 2.956990,
    (21, 'feature_36'): 1.032210,
    (21, 'feature_37'): 0.825186,
    (21, 'feature_38'): 1.457306,
    (21, 'feature_39'): 1.731384,
    (21, 'feature_40'): 0.835066,
    (21, 'feature_41'): 0.280338,
    (21, 'feature_42'): -0.781517,
    (21, 'feature_43'): 0.447516,
    (21, 'feature_44'): -0.591005,
    (21, 'feature_45'): 0.270392,
    (21, 'feature_46'): -1.179460,
    (21, 'feature_47'): -0.837837,
    (21, 'feature_48'): 0.328575,
    (21, 'feature_49'): 0.139205,
    (21, 'feature_50'): 1.924388,
    (21, 'feature_51'): 1.429835,
    (21, 'feature_52'): 0.687405,
    (21, 'feature_53'): -1.143321,
    (21, 'feature_54'): 0.162922,
    (21, 'feature_55'): -1.897149,
    (21, 'feature_56'): -0.042209,
    (21, 'feature_57'): -1.509235,
    (21, 'feature_58'): -1.096203,
    (21, 'feature_59'): 1.905004,
    (21, 'feature_60'): 0.527010,
    (21, 'feature_61'): -1.101531,
    (21, 'feature_62'): -0.475562,
    (21, 'feature_63'): -0.332480,
    (21, 'feature_64'): -0.270221,
    (21, 'feature_65'): 2.554265,
    (21, 'feature_66'): 0.379925,
    (21, 'feature_67'): 2.411573,
    (21, 'feature_68'): -0.043443,
    (21, 'feature_69'): 0.593250,
    (21, 'feature_70'): 0.954801,
    (21, 'feature_71'): 0.290414,
    (21, 'feature_72'): 1.459983,
    (21, 'feature_73'): 0.059806,
    (21, 'feature_74'): -0.055409,
    (21, 'feature_75'): 0.138859,
    (21, 'feature_76'): 0.040269,
    (21, 'feature_77'): 0.142225,
    (21, 'feature_78'): 0.025788,
    (22, 'feature_00'): 2.834406,
    (22, 'feature_01'): -0.971490,
    (22, 'feature_02'): 1.848975,
    (22, 'feature_03'): 2.231996,
    (22, 'feature_04'): 0.024626,
    (22, 'feature_05'): 0.795673,
    (22, 'feature_06'): 1.809209,
    (22, 'feature_07'): 2.410471,
    (22, 'feature_08'): 0.363069,
    (22, 'feature_09'): 30.000000,
    (22, 'feature_10'): 10.000000,
    (22, 'feature_11'): 13.000000,
    (22, 'feature_12'): 0.140474,
    (22, 'feature_13'): -0.131453,
    (22, 'feature_14'): -0.082959,
    (22, 'feature_15'): -0.545309,
    (22, 'feature_16'): -0.283483,
    (22, 'feature_17'): -0.532055,
    (22, 'feature_18'): 1.729547,
    (22, 'feature_19'): -0.057092,
    (22, 'feature_20'): -1.148794,
    (22, 'feature_21'): -0.161867,
    (22, 'feature_22'): -0.765677,
    (22, 'feature_23'): -0.206738,
    (22, 'feature_24'): -0.128137,
    (22, 'feature_25'): -0.842667,
    (22, 'feature_26'): 0.360339,
    (22, 'feature_27'): -0.320071,
    (22, 'feature_28'): -0.572710,
    (22, 'feature_29'): -0.551703,
    (22, 'feature_30'): -0.538859,
    (22, 'feature_31'): -0.131045,
    (22, 'feature_32'): 3.435568,
    (22, 'feature_33'): -0.744863,
    (22, 'feature_34'): 3.484949,
    (22, 'feature_35'): 3.358615,
    (22, 'feature_36'): -1.594062,
    (22, 'feature_37'): 1.045531,
    (22, 'feature_38'): 0.578297,
    (22, 'feature_39'): 0.646979,
    (22, 'feature_40'): 0.140078,
    (22, 'feature_41'): 1.373728,
    (22, 'feature_42'): 1.231501,
    (22, 'feature_43'): 0.164958,
    (22, 'feature_44'): -0.129047,
    (22, 'feature_45'): 0.067852,
    (22, 'feature_46'): -0.523722,
    (22, 'feature_47'): 0.114933,
    (22, 'feature_48'): -0.167794,
    (22, 'feature_49'): 1.363549,
    (22, 'feature_50'): 1.422291,
    (22, 'feature_51'): 0.097244,
    (22, 'feature_52'): 1.358746,
    (22, 'feature_53'): 2.379833,
    (22, 'feature_54'): 0.488043,
    (22, 'feature_55'): 1.909766,
    (22, 'feature_56'): 0.564014,
    (22, 'feature_57'): -1.205593,
    (22, 'feature_58'): 0.568102,
    (22, 'feature_59'): 0.643038,
    (22, 'feature_60'): 2.064394,
    (22, 'feature_61'): -1.101531,
    (22, 'feature_62'): -0.314942,
    (22, 'feature_63'): -0.225789,
    (22, 'feature_64'): -0.304565,
    (22, 'feature_65'): 2.459906,
    (22, 'feature_66'): -0.267286,
    (22, 'feature_67'): 0.164768,
    (22, 'feature_68'): -0.053435,
    (22, 'feature_69'): -0.185436,
    (22, 'feature_70'): 0.079167,
    (22, 'feature_71'): -0.108428,
    (22, 'feature_72'): 0.037226,
    (22, 'feature_73'): 0.134090,
    (22, 'feature_74'): 0.079512,
    (22, 'feature_75'): -0.049786,
    (22, 'feature_76'): -0.063217,
    (22, 'feature_77'): 0.247487,
    (22, 'feature_78'): 0.160977,
    (23, 'feature_00'): 2.117160,
    (23, 'feature_01'): -0.711313,
    (23, 'feature_02'): 1.922251,
    (23, 'feature_03'): 2.941116,
    (23, 'feature_04'): 0.703496,
    (23, 'feature_05'): 0.720659,
    (23, 'feature_06'): 1.556000,
    (23, 'feature_07'): 2.317007,
    (23, 'feature_08'): 0.243376,
    (23, 'feature_09'): 34.000000,
    (23, 'feature_10'): 4.000000,
    (23, 'feature_11'): 214.000000,
    (23, 'feature_12'): 1.097040,
    (23, 'feature_13'): 0.353393,
    (23, 'feature_14'): 0.281119,
    (23, 'feature_15'): -0.502186,
    (23, 'feature_16'): -0.614214,
    (23, 'feature_17'): -0.596483,
    (23, 'feature_18'): 2.005304,
    (23, 'feature_19'): -1.609560,
    (23, 'feature_20'): -0.176271,
    (23, 'feature_21'): -0.255326,
    (23, 'feature_22'): 0.080821,
    (23, 'feature_23'): 0.163571,
    (23, 'feature_24'): -0.084606,
    (23, 'feature_25'): -1.186233,
    (23, 'feature_26'): 1.790098,
    (23, 'feature_27'): 1.131597,
    (23, 'feature_28'): 0.729248,
    (23, 'feature_29'): -0.428183,
    (23, 'feature_30'): -0.505547,
    (23, 'feature_31'): -0.242358,
    (23, 'feature_32'): 3.000169,
    (23, 'feature_33'): -1.356090,
    (23, 'feature_34'): 2.831112,
    (23, 'feature_35'): 3.126695,
    (23, 'feature_36'): 0.211085,
    (23, 'feature_37'): 1.959888,
    (23, 'feature_38'): 0.818175,
    (23, 'feature_39'): 1.975085,
    (23, 'feature_40'): 0.628723,
    (23, 'feature_41'): 0.839092,
    (23, 'feature_42'): 1.065577,
    (23, 'feature_43'): -0.254557,
    (23, 'feature_44'): 0.356979,
    (23, 'feature_45'): 0.373734,
    (23, 'feature_46'): -1.720942,
    (23, 'feature_47'): 1.075989,
    (23, 'feature_48'): 1.263155,
    (23, 'feature_49'): 2.458218,
    (23, 'feature_50'): 1.883131,
    (23, 'feature_51'): 0.812777,
    (23, 'feature_52'): 1.203417,
    (23, 'feature_53'): 1.828498,
    (23, 'feature_54'): 0.440969,
    (23, 'feature_55'): 1.277164,
    (23, 'feature_56'): 0.990661,
    (23, 'feature_57'): -1.938705,
    (23, 'feature_58'): 1.113121,
    (23, 'feature_59'): 1.811035,
    (23, 'feature_60'): 3.247292,
    (23, 'feature_61'): -1.101531,
    (23, 'feature_62'): -0.492191,
    (23, 'feature_63'): -0.175507,
    (23, 'feature_64'): -0.303286,
    (23, 'feature_65'): 2.401801,
    (23, 'feature_66'): -1.531985,
    (23, 'feature_67'): 1.108738,
    (23, 'feature_68'): 0.047523,
    (23, 'feature_69'): 0.156593,
    (23, 'feature_70'): 1.035752,
    (23, 'feature_71'): 0.699813,
    (23, 'feature_72'): 0.390066,
    (23, 'feature_73'): -0.075733,
    (23, 'feature_74'): -0.267973,
    (23, 'feature_75'): -0.116324,
    (23, 'feature_76'): -0.178401,
    (23, 'feature_77'): 0.032641,
    (23, 'feature_78'): -0.090248,
    (24, 'feature_00'): 2.618003,
    (24, 'feature_01'): -0.764667,
    (24, 'feature_02'): 2.892787,
    (24, 'feature_03'): 2.189208,
    (24, 'feature_04'): 0.658402,
    (24, 'feature_05'): 0.547378,
    (24, 'feature_06'): 1.169915,
    (24, 'feature_07'): 1.778667,
    (24, 'feature_08'): 0.352484,
    (24, 'feature_09'): 14.000000,
    (24, 'feature_10'): 10.000000,
    (24, 'feature_11'): 40.000000,
    (24, 'feature_12'): 0.910694,
    (24, 'feature_13'): 1.880381,
    (24, 'feature_14'): 1.038154,
    (24, 'feature_15'): -0.715687,
    (24, 'feature_16'): -0.473468,
    (24, 'feature_17'): -0.462728,
    (24, 'feature_18'): 0.007770,
    (24, 'feature_19'): 4.436400,
    (24, 'feature_20'): 0.008324,
    (24, 'feature_21'): -0.171533,
    (24, 'feature_22'): -0.325619,
    (24, 'feature_23'): 0.183020,
    (24, 'feature_24'): 0.459248,
    (24, 'feature_25'): -0.069749,
    (24, 'feature_26'): -0.831014,
    (24, 'feature_27'): -0.644697,
    (24, 'feature_28'): -0.282557,
    (24, 'feature_29'): -0.636164,
    (24, 'feature_30'): -0.847914,
    (24, 'feature_31'): -0.146312,
    (24, 'feature_32'): 3.118166,
    (24, 'feature_33'): -1.140890,
    (24, 'feature_34'): 2.875343,
    (24, 'feature_35'): 3.174856,
    (24, 'feature_36'): -1.847669,
    (24, 'feature_37'): -2.016335,
    (24, 'feature_38'): -2.077608,
    (24, 'feature_39'): 1.176526,
    (24, 'feature_40'): 0.528832,
    (24, 'feature_41'): 0.797267,
    (24, 'feature_42'): 0.718825,
    (24, 'feature_43'): -0.048316,
    (24, 'feature_44'): 0.266109,
    (24, 'feature_45'): 1.836362,
    (24, 'feature_46'): 0.844204,
    (24, 'feature_47'): 0.669047,
    (24, 'feature_48'): -0.014109,
    (24, 'feature_49'): 0.747959,
    (24, 'feature_50'): 1.251550,
    (24, 'feature_51'): 0.152397,
    (24, 'feature_52'): 0.579733,
    (24, 'feature_53'): 0.632537,
    (24, 'feature_54'): -1.101145,
    (24, 'feature_55'): 0.081772,
    (24, 'feature_56'): 1.762420,
    (24, 'feature_57'): 1.575186,
    (24, 'feature_58'): 0.881640,
    (24, 'feature_59'): -0.111207,
    (24, 'feature_60'): 1.223176,
    (24, 'feature_61'): -1.101531,
    (24, 'feature_62'): -0.350155,
    (24, 'feature_63'): -0.418096,
    (24, 'feature_64'): -0.415535,
    (24, 'feature_65'): 0.183108,
    (24, 'feature_66'): 0.809196,
    (24, 'feature_67'): 0.596540,
    (24, 'feature_68'): 2.198485,
    (24, 'feature_69'): 0.665553,
    (24, 'feature_70'): 0.957064,
    (24, 'feature_71'): 0.818689,
    (24, 'feature_72'): 1.299827,
    (24, 'feature_73'): -0.152559,
    (24, 'feature_74'): -0.009907,
    (24, 'feature_75'): -0.098498,
    (24, 'feature_76'): 0.035121,
    (24, 'feature_77'): -0.105651,
    (24, 'feature_78'): 0.022888,
    (25, 'feature_00'): 2.340220,
    (25, 'feature_01'): -0.604505,
    (25, 'feature_02'): 2.705268,
    (25, 'feature_03'): 2.608655,
    (25, 'feature_04'): 0.248039,
    (25, 'feature_05'): 1.058375,
    (25, 'feature_06'): 1.688447,
    (25, 'feature_07'): 1.958356,
    (25, 'feature_08'): 0.399330,
    (25, 'feature_09'): 57.000000,
    (25, 'feature_10'): 1.000000,
    (25, 'feature_11'): 230.000000,
    (25, 'feature_12'): 0.433085,
    (25, 'feature_13'): 1.126564,
    (25, 'feature_14'): 0.407315,
    (25, 'feature_15'): -0.534305,
    (25, 'feature_16'): -0.493555,
    (25, 'feature_17'): -0.568392,
    (25, 'feature_18'): 2.676066,
    (25, 'feature_19'): 0.079733,
    (25, 'feature_20'): -0.086143,
    (25, 'feature_21'): 0.112843,
    (25, 'feature_22'): 0.403668,
    (25, 'feature_23'): 1.591298,
    (25, 'feature_24'): -0.170285,
    (25, 'feature_25'): -1.244043,
    (25, 'feature_26'): -0.779890,
    (25, 'feature_27'): -0.365266,
    (25, 'feature_28'): -0.391418,
    (25, 'feature_29'): -0.582926,
    (25, 'feature_30'): -0.508190,
    (25, 'feature_31'): 0.087313,
    (25, 'feature_32'): 2.848290,
    (25, 'feature_33'): -0.934151,
    (25, 'feature_34'): 3.277401,
    (25, 'feature_35'): 2.410307,
    (25, 'feature_36'): -0.668293,
    (25, 'feature_37'): 0.541567,
    (25, 'feature_38'): 0.120718,
    (25, 'feature_39'): 1.251210,
    (25, 'feature_40'): 0.151609,
    (25, 'feature_41'): 0.787561,
    (25, 'feature_42'): 0.718348,
    (25, 'feature_43'): -0.847209,
    (25, 'feature_44'): -0.516568,
    (25, 'feature_45'): 0.667202,
    (25, 'feature_46'): -1.531017,
    (25, 'feature_47'): 0.396583,
    (25, 'feature_48'): 0.602328,
    (25, 'feature_49'): 1.292190,
    (25, 'feature_50'): 0.625024,
    (25, 'feature_51'): 1.013867,
    (25, 'feature_52'): 1.358622,
    (25, 'feature_53'): 1.006312,
    (25, 'feature_54'): 0.424494,
    (25, 'feature_55'): 1.061097,
    (25, 'feature_56'): 0.656483,
    (25, 'feature_57'): -0.884379,
    (25, 'feature_58'): 0.859302,
    (25, 'feature_59'): 0.996386,
    (25, 'feature_60'): 1.426375,
    (25, 'feature_61'): -1.101531,
    (25, 'feature_62'): -0.487551,
    (25, 'feature_63'): -0.287804,
    (25, 'feature_64'): -0.445607,
    (25, 'feature_65'): 2.223632,
    (25, 'feature_66'): -0.202508,
    (25, 'feature_67'): 0.252938,
    (25, 'feature_68'): 0.117632,
    (25, 'feature_69'): -0.056761,
    (25, 'feature_70'): 0.627204,
    (25, 'feature_71'): 1.125836,
    (25, 'feature_72'): 0.900908,
    (25, 'feature_73'): -0.095817,
    (25, 'feature_74'): -0.092486,
    (25, 'feature_75'): -0.017675,
    (25, 'feature_76'): -0.029206,
    (25, 'feature_77'): -0.049177,
    (25, 'feature_78'): -0.036830,
    (26, 'feature_00'): 2.532538,
    (26, 'feature_01'): -0.541144,
    (26, 'feature_02'): 2.492719,
    (26, 'feature_03'): 2.198675,
    (26, 'feature_04'): -0.228492,
    (26, 'feature_05'): 0.812433,
    (26, 'feature_06'): 1.045924,
    (26, 'feature_07'): 1.971797,
    (26, 'feature_08'): 0.391897,
    (26, 'feature_09'): 26.000000,
    (26, 'feature_10'): 12.000000,
    (26, 'feature_11'): 158.000000,
    (26, 'feature_12'): 1.584755,
    (26, 'feature_13'): 1.042698,
    (26, 'feature_14'): 1.641423,
    (26, 'feature_15'): -0.382396,
    (26, 'feature_16'): -0.507798,
    (26, 'feature_17'): -0.491950,
    (26, 'feature_18'): 2.665463,
    (26, 'feature_19'): -0.565659,
    (26, 'feature_20'): -1.364399,
    (26, 'feature_21'): -0.161235,
    (26, 'feature_22'): -0.507549,
    (26, 'feature_23'): -0.374464,
    (26, 'feature_24'): -0.049864,
    (26, 'feature_25'): -0.918009,
    (26, 'feature_26'): -1.052620,
    (26, 'feature_27'): -0.930092,
    (26, 'feature_28'): -0.277262,
    (26, 'feature_29'): -0.835627,
    (26, 'feature_30'): -0.647298,
    (26, 'feature_31'): -0.147005,
    (26, 'feature_32'): 3.481039,
    (26, 'feature_33'): -0.450518,
    (26, 'feature_34'): 3.348833,
    (26, 'feature_35'): 2.807755,
    (26, 'feature_36'): -1.106108,
    (26, 'feature_37'): 0.259530,
    (26, 'feature_38'): 0.101016,
    (26, 'feature_39'): 1.448355,
    (26, 'feature_40'): 0.183281,
    (26, 'feature_41'): 1.147389,
    (26, 'feature_42'): 1.209693,
    (26, 'feature_43'): 0.310896,
    (26, 'feature_44'): 0.726939,
    (26, 'feature_45'): -0.072003,
    (26, 'feature_46'): -1.001416,
    (26, 'feature_47'): -0.378399,
    (26, 'feature_48'): 1.057302,
    (26, 'feature_49'): 2.210059,
    (26, 'feature_50'): 1.873396,
    (26, 'feature_51'): 1.409213,
    (26, 'feature_52'): 1.223163,
    (26, 'feature_53'): 1.986771,
    (26, 'feature_54'): 1.484391,
    (26, 'feature_55'): 2.031126,
    (26, 'feature_56'): 0.568793,
    (26, 'feature_57'): -1.585399,
    (26, 'feature_58'): -0.208900,
    (26, 'feature_59'): 1.494499,
    (26, 'feature_60'): 2.258863,
    (26, 'feature_61'): -1.101531,
    (26, 'feature_62'): -0.367285,
    (26, 'feature_63'): -0.214249,
    (26, 'feature_64'): -0.403089,
    (26, 'feature_65'): 2.515773,
    (26, 'feature_66'): -0.532529,
    (26, 'feature_67'): 0.857785,
    (26, 'feature_68'): 0.185112,
    (26, 'feature_69'): 0.501121,
    (26, 'feature_70'): 1.498836,
    (26, 'feature_71'): 1.158361,
    (26, 'feature_72'): 2.036570,
    (26, 'feature_73'): 2.669205,
    (26, 'feature_74'): 1.218873,
    (26, 'feature_75'): 2.503373,
    (26, 'feature_76'): 1.703673,
    (26, 'feature_77'): 1.827614,
    (26, 'feature_78'): 1.972112,
    (27, 'feature_00'): 2.479974,
    (27, 'feature_01'): -0.570381,
    (27, 'feature_02'): 2.731363,
    (27, 'feature_03'): 2.141496,
    (27, 'feature_04'): 0.280759,
    (27, 'feature_05'): 0.617193,
    (27, 'feature_06'): 1.244904,
    (27, 'feature_07'): 0.992152,
    (27, 'feature_08'): 0.189409,
    (27, 'feature_09'): 82.000000,
    (27, 'feature_10'): 3.000000,
    (27, 'feature_11'): 539.000000,
    (27, 'feature_12'): -0.237450,
    (27, 'feature_13'): 0.286864,
    (27, 'feature_14'): 0.025256,
    (27, 'feature_15'): -0.633825,
    (27, 'feature_16'): -0.597293,
    (27, 'feature_17'): -0.614579,
    (27, 'feature_18'): 0.139540,
    (27, 'feature_19'): 3.744738,
    (27, 'feature_20'): -0.976987,
    (27, 'feature_21'): 0.018075,
    (27, 'feature_22'): 1.342262,
    (27, 'feature_23'): 0.295671,
    (27, 'feature_24'): 1.614137,
    (27, 'feature_25'): 0.750667,
    (27, 'feature_26'): -0.998846,
    (27, 'feature_27'): 0.166219,
    (27, 'feature_28'): 0.208408,
    (27, 'feature_29'): -0.796773,
    (27, 'feature_30'): -0.807088,
    (27, 'feature_31'): 0.013520,
    (27, 'feature_32'): 3.153874,
    (27, 'feature_33'): -1.131275,
    (27, 'feature_34'): 2.740793,
    (27, 'feature_35'): 2.941266,
    (27, 'feature_36'): -1.507159,
    (27, 'feature_37'): -0.136996,
    (27, 'feature_38'): -0.253958,
    (27, 'feature_39'): 0.382926,
    (27, 'feature_40'): -0.180657,
    (27, 'feature_41'): 0.364264,
    (27, 'feature_42'): 0.917585,
    (27, 'feature_43'): -0.670035,
    (27, 'feature_44'): -0.062678,
    (27, 'feature_45'): 0.390105,
    (27, 'feature_46'): 0.505129,
    (27, 'feature_47'): 0.240441,
    (27, 'feature_48'): 0.067843,
    (27, 'feature_49'): 0.320272,
    (27, 'feature_50'): 0.831361,
    (27, 'feature_51'): 0.241031,
    (27, 'feature_52'): 0.168365,
    (27, 'feature_53'): 1.387926,
    (27, 'feature_54'): -0.374907,
    (27, 'feature_55'): 1.198004,
    (27, 'feature_56'): 0.774055,
    (27, 'feature_57'): 2.239745,
    (27, 'feature_58'): 0.785848,
    (27, 'feature_59'): 0.699457,
    (27, 'feature_60'): 0.686107,
    (27, 'feature_61'): -1.101531,
    (27, 'feature_62'): -0.431342,
    (27, 'feature_63'): -0.288966,
    (27, 'feature_64'): -0.320378,
    (27, 'feature_65'): 0.192498,
    (27, 'feature_66'): 1.735018,
    (27, 'feature_67'): -0.276520,
    (27, 'feature_68'): -0.043241,
    (27, 'feature_69'): -0.263914,
    (27, 'feature_70'): -0.061066,
    (27, 'feature_71'): 0.325867,
    (27, 'feature_72'): 0.218496,
    (27, 'feature_73'): -0.130882,
    (27, 'feature_74'): -0.083665,
    (27, 'feature_75'): 0.222558,
    (27, 'feature_76'): 0.318667,
    (27, 'feature_77'): 0.043913,
    (27, 'feature_78'): 0.070607,
    (28, 'feature_00'): 2.685024,
    (28, 'feature_01'): -0.722533,
    (28, 'feature_02'): 2.293697,
    (28, 'feature_03'): 2.229603,
    (28, 'feature_04'): 0.611679,
    (28, 'feature_05'): 0.948255,
    (28, 'feature_06'): 1.322343,
    (28, 'feature_07'): 2.517851,
    (28, 'feature_08'): 0.334319,
    (28, 'feature_09'): 9.000000,
    (28, 'feature_10'): 3.000000,
    (28, 'feature_11'): 25.000000,
    (28, 'feature_12'): 0.738704,
    (28, 'feature_13'): 0.664605,
    (28, 'feature_14'): 1.096888,
    (28, 'feature_15'): -0.686848,
    (28, 'feature_16'): -0.510505,
    (28, 'feature_17'): -0.566731,
    (28, 'feature_18'): 2.455940,
    (28, 'feature_19'): 0.686706,
    (28, 'feature_20'): -0.708694,
    (28, 'feature_21'): -0.243013,
    (28, 'feature_22'): -0.116336,
    (28, 'feature_23'): 0.152671,
    (28, 'feature_24'): -0.068289,
    (28, 'feature_25'): -0.665109,
    (28, 'feature_26'): -1.007817,
    (28, 'feature_27'): 0.173513,
    (28, 'feature_28'): 1.173667,
    (28, 'feature_29'): -0.639638,
    (28, 'feature_30'): -0.645127,
    (28, 'feature_31'): -0.311854,
    (28, 'feature_32'): 2.949332,
    (28, 'feature_33'): -0.962816,
    (28, 'feature_34'): 3.085716,
    (28, 'feature_35'): 2.894101,
    (28, 'feature_36'): 0.746068,
    (28, 'feature_37'): 1.710146,
    (28, 'feature_38'): 2.064749,
    (28, 'feature_39'): 1.368691,
    (28, 'feature_40'): 0.151333,
    (28, 'feature_41'): 1.338426,
    (28, 'feature_42'): 0.406776,
    (28, 'feature_43'): -0.492382,
    (28, 'feature_44'): 0.328067,
    (28, 'feature_45'): 0.579186,
    (28, 'feature_46'): -1.903498,
    (28, 'feature_47'): 0.633887,
    (28, 'feature_48'): -0.027259,
    (28, 'feature_49'): 1.063499,
    (28, 'feature_50'): 1.130664,
    (28, 'feature_51'): -1.138751,
    (28, 'feature_52'): 0.440761,
    (28, 'feature_53'): 1.286464,
    (28, 'feature_54'): 0.010817,
    (28, 'feature_55'): 0.153788,
    (28, 'feature_56'): 0.118026,
    (28, 'feature_57'): -1.708484,
    (28, 'feature_58'): 0.608934,
    (28, 'feature_59'): -0.378395,
    (28, 'feature_60'): 1.339864,
    (28, 'feature_61'): -1.101531,
    (28, 'feature_62'): -0.478276,
    (28, 'feature_63'): -0.241481,
    (28, 'feature_64'): -0.361363,
    (28, 'feature_65'): 2.233943,
    (28, 'feature_66'): -0.027177,
    (28, 'feature_67'): 1.052089,
    (28, 'feature_68'): 0.640855,
    (28, 'feature_69'): 0.424937,
    (28, 'feature_70'): 1.753137,
    (28, 'feature_71'): 0.824213,
    (28, 'feature_72'): 0.790969,
    (28, 'feature_73'): 0.129109,
    (28, 'feature_74'): 0.046176,
    (28, 'feature_75'): -0.114208,
    (28, 'feature_76'): -0.212878,
    (28, 'feature_77'): 0.087698,
    (28, 'feature_78'): 0.022628,
    (29, 'feature_00'): 2.947267,
    (29, 'feature_01'): -1.026029,
    (29, 'feature_02'): 2.586358,
    (29, 'feature_03'): 2.444806,
    (29, 'feature_04'): 0.112324,
    (29, 'feature_05'): 0.924211,
    (29, 'feature_06'): 1.028334,
    (29, 'feature_07'): 1.808578,
    (29, 'feature_08'): 0.235700,
    (29, 'feature_09'): 57.000000,
    (29, 'feature_10'): 1.000000,
    (29, 'feature_11'): 336.000000,
    (29, 'feature_12'): 0.236979,
    (29, 'feature_13'): 0.608571,
    (29, 'feature_14'): 0.285737,
    (29, 'feature_15'): -0.979101,
    (29, 'feature_16'): -0.575093,
    (29, 'feature_17'): -0.651753,
    (29, 'feature_18'): 2.560922,
    (29, 'feature_19'): -0.430278,
    (29, 'feature_20'): -0.396807,
    (29, 'feature_21'): 0.017042,
    (29, 'feature_22'): -0.465209,
    (29, 'feature_23'): -0.542337,
    (29, 'feature_24'): 1.019189,
    (29, 'feature_25'): 0.547484,
    (29, 'feature_26'): 0.340598,
    (29, 'feature_27'): -0.919110,
    (29, 'feature_28'): -1.569679,
    (29, 'feature_29'): -0.789114,
    (29, 'feature_30'): -0.853733,
    (29, 'feature_31'): 0.022431,
    (29, 'feature_32'): 3.585600,
    (29, 'feature_33'): -0.265938,
    (29, 'feature_34'): 2.998845,
    (29, 'feature_35'): 2.582362,
    (29, 'feature_36'): 0.864101,
    (29, 'feature_37'): 0.567645,
    (29, 'feature_38'): 0.414774,
    (29, 'feature_39'): 0.536740,
    (29, 'feature_40'): -0.864473,
    (29, 'feature_41'): 0.008222,
    (29, 'feature_42'): 0.998071,
    (29, 'feature_43'): -0.190916,
    (29, 'feature_44'): -0.030016,
    (29, 'feature_45'): -0.745488,
    (29, 'feature_46'): -0.690772,
    (29, 'feature_47'): 0.441729,
    (29, 'feature_48'): 0.000591,
    (29, 'feature_49'): 0.361442,
    (29, 'feature_50'): 0.612492,
    (29, 'feature_51'): 0.533941,
    (29, 'feature_52'): 0.077449,
    (29, 'feature_53'): 1.826184,
    (29, 'feature_54'): 0.416273,
    (29, 'feature_55'): 1.348086,
    (29, 'feature_56'): -0.200461,
    (29, 'feature_57'): -1.343582,
    (29, 'feature_58'): 0.509794,
    (29, 'feature_59'): 0.721500,
    (29, 'feature_60'): 0.618154,
    (29, 'feature_61'): -1.101531,
    (29, 'feature_62'): -0.537916,
    (29, 'feature_63'): -0.322354,
    (29, 'feature_64'): -0.371880,
    (29, 'feature_65'): 2.573768,
    (29, 'feature_66'): -0.287869,
    (29, 'feature_67'): 0.121696,
    (29, 'feature_68'): 0.178009,
    (29, 'feature_69'): -0.061740,
    (29, 'feature_70'): 0.293774,
    (29, 'feature_71'): 0.965394,
    (29, 'feature_72'): 0.882430,
    (29, 'feature_73'): 0.512395,
    (29, 'feature_74'): 0.408428,
    (29, 'feature_75'): 0.018710,
    (29, 'feature_76'): -0.035679,
    (29, 'feature_77'): 0.183121,
    (29, 'feature_78'): 0.095118,
    (30, 'feature_00'): 2.056912,
    (30, 'feature_01'): -0.632975,
    (30, 'feature_02'): 2.301148,
    (30, 'feature_03'): 2.369961,
    (30, 'feature_04'): 0.212745,
    (30, 'feature_05'): 0.364430,
    (30, 'feature_06'): 0.868055,
    (30, 'feature_07'): 1.655553,
    (30, 'feature_08'): 0.194803,
    (30, 'feature_09'): 81.000000,
    (30, 'feature_10'): 2.000000,
    (30, 'feature_11'): 534.000000,
    (30, 'feature_12'): -0.333754,
    (30, 'feature_13'): -0.130268,
    (30, 'feature_14'): -0.466275,
    (30, 'feature_15'): -0.741652,
    (30, 'feature_16'): -0.649031,
    (30, 'feature_17'): -0.670350,
    (30, 'feature_18'): -2.090105,
    (30, 'feature_19'): 1.310893,
    (30, 'feature_20'): -0.540160,
    (30, 'feature_21'): 0.164429,
    (30, 'feature_22'): 0.158168,
    (30, 'feature_23'): 0.087220,
    (30, 'feature_24'): 2.037113,
    (30, 'feature_25'): 0.847346,
    (30, 'feature_26'): -1.520364,
    (30, 'feature_27'): -0.871927,
    (30, 'feature_28'): -0.112472,
    (30, 'feature_29'): -0.763437,
    (30, 'feature_30'): -1.251311,
    (30, 'feature_31'): 0.160000,
    (30, 'feature_32'): 3.311137,
    (30, 'feature_33'): -0.600691,
    (30, 'feature_34'): 3.500964,
    (30, 'feature_35'): 2.995498,
    (30, 'feature_36'): -0.524425,
    (30, 'feature_37'): -0.279560,
    (30, 'feature_38'): -0.257479,
    (30, 'feature_39'): 0.052615,
    (30, 'feature_40'): 0.568726,
    (30, 'feature_41'): 0.405350,
    (30, 'feature_42'): 1.640010,
    (30, 'feature_43'): 0.671941,
    (30, 'feature_44'): 0.955440,
    (30, 'feature_45'): -1.158496,
    (30, 'feature_46'): -0.160249,
    (30, 'feature_47'): -0.070044,
    (30, 'feature_48'): 0.449414,
    (30, 'feature_49'): 0.292899,
    (30, 'feature_50'): -0.387835,
    (30, 'feature_51'): -0.850615,
    (30, 'feature_52'): 0.126507,
    (30, 'feature_53'): 1.663030,
    (30, 'feature_54'): 0.274152,
    (30, 'feature_55'): 0.893509,
    (30, 'feature_56'): -0.470135,
    (30, 'feature_57'): 0.312032,
    (30, 'feature_58'): -0.078076,
    (30, 'feature_59'): 0.350034,
    (30, 'feature_60'): 0.624537,
    (30, 'feature_61'): -1.101531,
    (30, 'feature_62'): -0.554068,
    (30, 'feature_63'): -0.471503,
    (30, 'feature_64'): -0.304775,
    (30, 'feature_65'): -1.596309,
    (30, 'feature_66'): 0.475938,
    (30, 'feature_67'): -0.384371,
    (30, 'feature_68'): -0.125588,
    (30, 'feature_69'): -0.444805,
    (30, 'feature_70'): -0.399089,
    (30, 'feature_71'): -0.161392,
    (30, 'feature_72'): -0.427888,
    (30, 'feature_73'): -0.346550,
    (30, 'feature_74'): -0.205084,
    (30, 'feature_75'): -0.233613,
    (30, 'feature_76'): -0.302901,
    (30, 'feature_77'): -0.309356,
    (30, 'feature_78'): -0.241081,
    (31, 'feature_00'): 2.774562,
    (31, 'feature_01'): -0.773040,
    (31, 'feature_02'): 2.010962,
    (31, 'feature_03'): 2.408067,
    (31, 'feature_04'): 0.031825,
    (31, 'feature_05'): 0.481788,
    (31, 'feature_06'): 0.826586,
    (31, 'feature_07'): 1.023392,
    (31, 'feature_08'): 0.134700,
    (31, 'feature_09'): 44.000000,
    (31, 'feature_10'): 3.000000,
    (31, 'feature_11'): 24.000000,
    (31, 'feature_12'): 6.907330,
    (31, 'feature_13'): 4.679749,
    (31, 'feature_14'): 4.393652,
    (31, 'feature_15'): -0.752877,
    (31, 'feature_16'): -0.713661,
    (31, 'feature_17'): -0.674699,
    (31, 'feature_18'): 1.058324,
    (31, 'feature_19'): 1.438036,
    (31, 'feature_20'): -0.456826,
    (31, 'feature_21'): -0.114990,
    (31, 'feature_22'): -0.665442,
    (31, 'feature_23'): -0.559475,
    (31, 'feature_24'): 2.510873,
    (31, 'feature_25'): 1.129548,
    (31, 'feature_26'): 0.201498,
    (31, 'feature_27'): -0.945420,
    (31, 'feature_28'): -1.447688,
    (31, 'feature_29'): -0.810108,
    (31, 'feature_30'): -0.701579,
    (31, 'feature_31'): -0.117847,
    (31, 'feature_32'): 3.255830,
    (31, 'feature_33'): -1.494415,
    (31, 'feature_34'): 3.104509,
    (31, 'feature_35'): 3.151791,
    (31, 'feature_36'): -2.093114,
    (31, 'feature_37'): 0.386071,
    (31, 'feature_38'): 0.425080,
    (31, 'feature_39'): 1.488326,
    (31, 'feature_40'): 0.024402,
    (31, 'feature_41'): -0.248305,
    (31, 'feature_42'): 0.773220,
    (31, 'feature_43'): -0.280021,
    (31, 'feature_44'): -0.261906,
    (31, 'feature_45'): 1.758451,
    (31, 'feature_46'): -1.524529,
    (31, 'feature_47'): 0.395392,
    (31, 'feature_48'): -0.017066,
    (31, 'feature_49'): 0.560670,
    (31, 'feature_50'): 2.087900,
    (31, 'feature_51'): -0.058643,
    (31, 'feature_52'): -0.117105,
    (31, 'feature_53'): 1.816323,
    (31, 'feature_54'): 0.273408,
    (31, 'feature_55'): 1.264519,
    (31, 'feature_56'): 1.522826,
    (31, 'feature_57'): -0.784295,
    (31, 'feature_58'): 0.925364,
    (31, 'feature_59'): 0.204506,
    (31, 'feature_60'): 1.383907,
    (31, 'feature_61'): -1.101531,
    (31, 'feature_62'): -0.376347,
    (31, 'feature_63'): -0.218164,
    (31, 'feature_64'): -0.354202,
    (31, 'feature_65'): 0.380918,
    (31, 'feature_66'): -1.176435,
    (31, 'feature_67'): 4.170466,
    (31, 'feature_68'): 3.961658,
    (31, 'feature_69'): 4.545374,
    (31, 'feature_70'): 6.716705,
    (31, 'feature_71'): 1.470822,
    (31, 'feature_72'): 3.905352,
    (31, 'feature_73'): 5.870592,
    (31, 'feature_74'): 3.472188,
    (31, 'feature_75'): 6.326633,
    (31, 'feature_76'): 5.335399,
    (31, 'feature_77'): 4.475570,
    (31, 'feature_78'): 4.090820,
    (32, 'feature_00'): 2.785486,
    (32, 'feature_01'): -0.702068,
    (32, 'feature_02'): 2.321103,
    (32, 'feature_03'): 2.250055,
    (32, 'feature_04'): 0.043770,
    (32, 'feature_05'): 0.543844,
    (32, 'feature_06'): 1.426622,
    (32, 'feature_07'): 2.278748,
    (32, 'feature_08'): 0.226032,
    (32, 'feature_09'): 26.000000,
    (32, 'feature_10'): 12.000000,
    (32, 'feature_11'): 158.000000,
    (32, 'feature_12'): -0.051345,
    (32, 'feature_13'): 0.394760,
    (32, 'feature_14'): -0.050388,
    (32, 'feature_15'): -0.379724,
    (32, 'feature_16'): -0.538526,
    (32, 'feature_17'): -0.819018,
    (32, 'feature_18'): 1.355522,
    (32, 'feature_19'): -0.752731,
    (32, 'feature_20'): 0.135453,
    (32, 'feature_21'): 0.061558,
    (32, 'feature_22'): -0.086540,
    (32, 'feature_23'): 0.207423,
    (32, 'feature_24'): 0.811900,
    (32, 'feature_25'): 0.249649,
    (32, 'feature_26'): 2.089990,
    (32, 'feature_27'): -1.269443,
    (32, 'feature_28'): -1.148063,
    (32, 'feature_29'): -0.417585,
    (32, 'feature_30'): -0.517746,
    (32, 'feature_31'): 0.076395,
    (32, 'feature_32'): 2.777242,
    (32, 'feature_33'): 0.468905,
    (32, 'feature_34'): 3.299363,
    (32, 'feature_35'): 2.932701,
    (32, 'feature_36'): -1.208795,
    (32, 'feature_37'): 0.439010,
    (32, 'feature_38'): 0.445466,
    (32, 'feature_39'): 0.826008,
    (32, 'feature_40'): 1.179291,
    (32, 'feature_41'): 0.498168,
    (32, 'feature_42'): 0.502105,
    (32, 'feature_43'): -0.613862,
    (32, 'feature_44'): -0.013670,
    (32, 'feature_45'): 0.498438,
    (32, 'feature_46'): -0.374323,
    (32, 'feature_47'): 0.429941,
    (32, 'feature_48'): 0.377404,
    (32, 'feature_49'): 1.087875,
    (32, 'feature_50'): 1.638834,
    (32, 'feature_51'): 0.618349,
    (32, 'feature_52'): -0.094321,
    (32, 'feature_53'): 1.484090,
    (32, 'feature_54'): 0.086542,
    (32, 'feature_55'): -0.122576,
    (32, 'feature_56'): 0.364664,
    (32, 'feature_57'): -0.248193,
    (32, 'feature_58'): 0.804460,
    (32, 'feature_59'): -0.234085,
    (32, 'feature_60'): 0.743950,
    (32, 'feature_61'): -1.101531,
    (32, 'feature_62'): -0.392961,
    (32, 'feature_63'): -0.308081,
    (32, 'feature_64'): -0.438358,
    (32, 'feature_65'): 2.226403,
    (32, 'feature_66'): -0.795969,
    (32, 'feature_67'): -0.163350,
    (32, 'feature_68'): 0.244001,
    (32, 'feature_69'): -0.152271,
    (32, 'feature_70'): -0.002686,
    (32, 'feature_71'): 0.205423,
    (32, 'feature_72'): 0.175455,
    (32, 'feature_73'): -0.158127,
    (32, 'feature_74'): -0.308199,
    (32, 'feature_75'): -0.153135,
    (32, 'feature_76'): -0.247434,
    (32, 'feature_77'): -0.347808,
    (32, 'feature_78'): -0.267499,
    (33, 'feature_00'): 2.294896,
    (33, 'feature_01'): -0.606888,
    (33, 'feature_02'): 3.003242,
    (33, 'feature_03'): 2.757708,
    (33, 'feature_04'): 0.408036,
    (33, 'feature_05'): 0.813578,
    (33, 'feature_06'): 1.955435,
    (33, 'feature_07'): 2.054828,
    (33, 'feature_08'): 0.343129,
    (33, 'feature_09'): 11.000000,
    (33, 'feature_10'): 7.000000,
    (33, 'feature_11'): 76.000000,
    (33, 'feature_12'): 0.219371,
    (33, 'feature_13'): 0.146427,
    (33, 'feature_14'): 0.125186,
    (33, 'feature_15'): -0.631374,
    (33, 'feature_16'): -0.512654,
    (33, 'feature_17'): -0.514075,
    (33, 'feature_18'): -2.428527,
    (33, 'feature_19'): -0.500306,
    (33, 'feature_20'): 0.512593,
    (33, 'feature_21'): -0.170628,
    (33, 'feature_22'): -0.250818,
    (33, 'feature_23'): 0.004504,
    (33, 'feature_24'): 0.390548,
    (33, 'feature_25'): -0.117636,
    (33, 'feature_26'): -0.193823,
    (33, 'feature_27'): 0.242282,
    (33, 'feature_28'): 0.031815,
    (33, 'feature_29'): -0.540262,
    (33, 'feature_30'): -0.709511,
    (33, 'feature_31'): -0.244782,
    (33, 'feature_32'): 3.195381,
    (33, 'feature_33'): -0.567780,
    (33, 'feature_34'): 2.995459,
    (33, 'feature_35'): 2.987542,
    (33, 'feature_36'): -0.868297,
    (33, 'feature_37'): -0.606492,
    (33, 'feature_38'): -1.099354,
    (33, 'feature_39'): 0.422453,
    (33, 'feature_40'): 0.494661,
    (33, 'feature_41'): 0.737259,
    (33, 'feature_42'): 0.438349,
    (33, 'feature_43'): 0.310990,
    (33, 'feature_44'): -0.770878,
    (33, 'feature_45'): -0.019925,
    (33, 'feature_46'): -0.185466,
    (33, 'feature_47'): -0.083969,
    (33, 'feature_48'): 0.006779,
    (33, 'feature_49'): 0.029664,
    (33, 'feature_50'): 0.131541,
    (33, 'feature_51'): -0.243334,
    (33, 'feature_52'): -0.241919,
    (33, 'feature_53'): 1.106015,
    (33, 'feature_54'): 0.737484,
    (33, 'feature_55'): 0.474197,
    (33, 'feature_56'): 0.144491,
    (33, 'feature_57'): 0.206050,
    (33, 'feature_58'): -0.022370,
    (33, 'feature_59'): 0.075682,
    (33, 'feature_60'): 0.164218,
    (33, 'feature_61'): -1.101531,
    (33, 'feature_62'): -0.361522,
    (33, 'feature_63'): -0.218588,
    (33, 'feature_64'): -0.356637,
    (33, 'feature_65'): -1.692127,
    (33, 'feature_66'): -0.678026,
    (33, 'feature_67'): 0.080329,
    (33, 'feature_68'): 0.011328,
    (33, 'feature_69'): 0.014887,
    (33, 'feature_70'): 0.344276,
    (33, 'feature_71'): 0.341116,
    (33, 'feature_72'): 0.314163,
    (33, 'feature_73'): -0.237698,
    (33, 'feature_74'): -0.237147,
    (33, 'feature_75'): -0.275476,
    (33, 'feature_76'): -0.197366,
    (33, 'feature_77'): -0.269815,
    (33, 'feature_78'): -0.191532,
    (34, 'feature_00'): 2.525160,
    (34, 'feature_01'): -0.721981,
    (34, 'feature_02'): 2.544025,
    (34, 'feature_03'): 2.477615,
    (34, 'feature_04'): 0.417557,
    (34, 'feature_05'): 0.785812,
    (34, 'feature_06'): 1.117796,
    (34, 'feature_07'): 2.199436,
    (34, 'feature_08'): 0.415427,
    (34, 'feature_09'): 42.000000,
    (34, 'feature_10'): 5.000000,
    (34, 'feature_11'): 150.000000,
    (34, 'feature_12'): 0.804403,
    (34, 'feature_13'): 1.157257,
    (34, 'feature_14'): 1.031543,
    (34, 'feature_15'): -0.671189,
    (34, 'feature_16'): -0.328600,
    (34, 'feature_17'): -0.486132,
    (34, 'feature_18'): 1.730176,
    (34, 'feature_19'): -0.006173,
    (34, 'feature_20'): -0.001144,
    (34, 'feature_21'): -0.213062,
    (34, 'feature_22'): 0.932618,
    (34, 'feature_23'): 1.367338,
    (34, 'feature_24'): -0.238197,
    (34, 'feature_25'): -0.692615,
    (34, 'feature_26'): -0.121163,
    (34, 'feature_27'): 1.090798,
    (34, 'feature_28'): 1.444294,
    (34, 'feature_29'): -0.675626,
    (34, 'feature_30'): -1.013264,
    (34, 'feature_31'): -0.242888,
    (34, 'feature_32'): 3.427639,
    (34, 'feature_33'): -0.958278,
    (34, 'feature_34'): 3.139836,
    (34, 'feature_35'): 3.416278,
    (34, 'feature_36'): -1.655316,
    (34, 'feature_37'): -0.599440,
    (34, 'feature_38'): -0.932876,
    (34, 'feature_39'): 2.493458,
    (34, 'feature_40'): 0.969462,
    (34, 'feature_41'): 1.102016,
    (34, 'feature_42'): 0.158982,
    (34, 'feature_43'): -0.496177,
    (34, 'feature_44'): 0.036177,
    (34, 'feature_45'): 1.309866,
    (34, 'feature_46'): 0.828025,
    (34, 'feature_47'): 1.577955,
    (34, 'feature_48'): 1.040802,
    (34, 'feature_49'): 1.255398,
    (34, 'feature_50'): 2.577441,
    (34, 'feature_51'): 0.057455,
    (34, 'feature_52'): 0.953005,
    (34, 'feature_53'): 1.377051,
    (34, 'feature_54'): -0.396358,
    (34, 'feature_55'): 0.520262,
    (34, 'feature_56'): 1.179617,
    (34, 'feature_57'): 1.127657,
    (34, 'feature_58'): 2.231928,
    (34, 'feature_59'): 0.614652,
    (34, 'feature_60'): 2.412886,
    (34, 'feature_61'): -1.101531,
    (34, 'feature_62'): -0.384833,
    (34, 'feature_63'): -0.275818,
    (34, 'feature_64'): -0.408040,
    (34, 'feature_65'): 2.427115,
    (34, 'feature_66'): -0.108427,
    (34, 'feature_67'): 0.739734,
    (34, 'feature_68'): 0.830205,
    (34, 'feature_69'): 0.366287,
    (34, 'feature_70'): 1.333250,
    (34, 'feature_71'): 1.075499,
    (34, 'feature_72'): 1.798264,
    (34, 'feature_73'): -0.183443,
    (34, 'feature_74'): -0.190222,
    (34, 'feature_75'): 0.234211,
    (34, 'feature_76'): 0.347142,
    (34, 'feature_77'): -0.044463,
    (34, 'feature_78'): 0.016936,
    (35, 'feature_00'): 1.857906,
    (35, 'feature_01'): -0.790646,
    (35, 'feature_02'): 2.745439,
    (35, 'feature_03'): 2.339877,
    (35, 'feature_04'): 0.845065,
    (35, 'feature_05'): 0.651370,
    (35, 'feature_06'): 1.180301,
    (35, 'feature_07'): 1.966379,
    (35, 'feature_08'): 0.321543,
    (35, 'feature_09'): 25.000000,
    (35, 'feature_10'): 7.000000,
    (35, 'feature_11'): 195.000000,
    (35, 'feature_12'): -0.075294,
    (35, 'feature_13'): -0.152726,
    (35, 'feature_14'): -0.204170,
    (35, 'feature_15'): -0.421137,
    (35, 'feature_16'): 0.217080,
    (35, 'feature_17'): -0.258775,
    (35, 'feature_18'): 1.874978,
    (35, 'feature_19'): 0.199880,
    (35, 'feature_20'): -0.199219,
    (35, 'feature_21'): -0.125619,
    (35, 'feature_22'): -1.004547,
    (35, 'feature_23'): -0.051933,
    (35, 'feature_24'): 0.450905,
    (35, 'feature_25'): 0.009246,
    (35, 'feature_26'): 0.164127,
    (35, 'feature_27'): -0.939974,
    (35, 'feature_28'): -1.143421,
    (35, 'feature_29'): -0.320071,
    (35, 'feature_30'): -0.379835,
    (35, 'feature_31'): -0.142429,
    (35, 'feature_32'): 3.862469,
    (35, 'feature_33'): -1.451786,
    (35, 'feature_34'): 3.477489,
    (35, 'feature_35'): 2.861663,
    (35, 'feature_36'): 0.763459,
    (35, 'feature_37'): 0.075972,
    (35, 'feature_38'): -0.119677,
    (35, 'feature_39'): 0.626035,
    (35, 'feature_40'): 0.148815,
    (35, 'feature_41'): 0.653281,
    (35, 'feature_42'): 0.059313,
    (35, 'feature_43'): -0.845099,
    (35, 'feature_44'): 0.098528,
    (35, 'feature_45'): 0.409564,
    (35, 'feature_46'): -0.675728,
    (35, 'feature_47'): -0.011334,
    (35, 'feature_48'): 0.930534,
    (35, 'feature_49'): 0.831980,
    (35, 'feature_50'): 0.808955,
    (35, 'feature_51'): 0.219276,
    (35, 'feature_52'): -0.315776,
    (35, 'feature_53'): 0.687755,
    (35, 'feature_54'): -1.189577,
    (35, 'feature_55'): 0.180146,
    (35, 'feature_56'): -0.175486,
    (35, 'feature_57'): -1.604350,
    (35, 'feature_58'): -0.209283,
    (35, 'feature_59'): 0.249847,
    (35, 'feature_60'): 0.288816,
    (35, 'feature_61'): -1.101531,
    (35, 'feature_62'): -0.343868,
    (35, 'feature_63'): -0.253991,
    (35, 'feature_64'): -0.278832,
    (35, 'feature_65'): 2.050639,
    (35, 'feature_66'): -0.059506,
    (35, 'feature_67'): -0.029396,
    (35, 'feature_68'): -0.101381,
    (35, 'feature_69'): -0.187759,
    (35, 'feature_70'): -0.180839,
    (35, 'feature_71'): -0.086100,
    (35, 'feature_72'): -0.153405,
    (35, 'feature_73'): -0.196077,
    (35, 'feature_74'): -0.175292,
    (35, 'feature_75'): 1.045780,
    (35, 'feature_76'): 0.739733,
    (35, 'feature_77'): 0.033720,
    (35, 'feature_78'): 0.050860,
    (36, 'feature_00'): 2.515527,
    (36, 'feature_01'): -0.672298,
    (36, 'feature_02'): 2.289250,
    (36, 'feature_03'): 2.521592,
    (36, 'feature_04'): 0.255077,
    (36, 'feature_05'): 0.919892,
    (36, 'feature_06'): 1.172018,
    (36, 'feature_07'): 2.180496,
    (36, 'feature_08'): 0.248460,
    (36, 'feature_09'): 49.000000,
    (36, 'feature_10'): 7.000000,
    (36, 'feature_11'): 297.000000,
    (36, 'feature_12'): 1.026715,
    (36, 'feature_13'): -0.096892,
    (36, 'feature_14'): 0.224309,
    (36, 'feature_15'): -0.528109,
    (36, 'feature_16'): -0.704952,
    (36, 'feature_17'): -0.704818,
    (36, 'feature_18'): 2.312482,
    (36, 'feature_19'): 0.328040,
    (36, 'feature_20'): -0.108193,
    (36, 'feature_21'): 0.089721,
    (36, 'feature_22'): -0.945684,
    (36, 'feature_23'): -0.244173,
    (36, 'feature_24'): 0.205989,
    (36, 'feature_25'): -0.357343,
    (36, 'feature_26'): -0.367174,
    (36, 'feature_27'): -1.014297,
    (36, 'feature_28'): -1.110750,
    (36, 'feature_29'): -0.580242,
    (36, 'feature_30'): -0.400568,
    (36, 'feature_31'): 0.079240,
    (36, 'feature_32'): 2.397877,
    (36, 'feature_33'): -0.637258,
    (36, 'feature_34'): 3.260638,
    (36, 'feature_35'): 3.046786,
    (36, 'feature_36'): 0.440965,
    (36, 'feature_37'): 0.234842,
    (36, 'feature_38'): -0.175580,
    (36, 'feature_39'): 1.022406,
    (36, 'feature_40'): -0.500069,
    (36, 'feature_41'): 2.071033,
    (36, 'feature_42'): 0.413488,
    (36, 'feature_43'): -0.450016,
    (36, 'feature_44'): -0.156616,
    (36, 'feature_45'): -0.253755,
    (36, 'feature_46'): -0.769588,
    (36, 'feature_47'): 0.066086,
    (36, 'feature_48'): 0.047826,
    (36, 'feature_49'): 1.713707,
    (36, 'feature_50'): 0.772772,
    (36, 'feature_51'): -0.549192,
    (36, 'feature_52'): 1.338474,
    (36, 'feature_53'): 0.933568,
    (36, 'feature_54'): 0.032978,
    (36, 'feature_55'): -0.519118,
    (36, 'feature_56'): -0.290343,
    (36, 'feature_57'): -0.806786,
    (36, 'feature_58'): 0.106295,
    (36, 'feature_59'): 0.183461,
    (36, 'feature_60'): 1.830421,
    (36, 'feature_61'): -1.101531,
    (36, 'feature_62'): -0.341991,
    (36, 'feature_63'): -0.249132,
    (36, 'feature_64'): -0.343650,
    (36, 'feature_65'): 2.251358,
    (36, 'feature_66'): 0.601888,
    (36, 'feature_67'): 1.035051,
    (36, 'feature_68'): -0.283241,
    (36, 'feature_69'): 0.107244,
    (36, 'feature_70'): 0.860160,
    (36, 'feature_71'): 0.024223,
    (36, 'feature_72'): 0.374852,
    (36, 'feature_73'): -0.220933,
    (36, 'feature_74'): -0.161584,
    (36, 'feature_75'): 0.032771,
    (36, 'feature_76'): 0.036888,
    (36, 'feature_77'): 0.168908,
    (36, 'feature_78'): 0.152333,
    (37, 'feature_00'): 2.663298,
    (37, 'feature_01'): -0.889112,
    (37, 'feature_02'): 2.313155,
    (37, 'feature_03'): 3.101428,
    (37, 'feature_04'): 0.324454,
    (37, 'feature_05'): 0.618944,
    (37, 'feature_06'): 1.185663,
    (37, 'feature_07'): 1.599724,
    (37, 'feature_08'): 0.319719,
    (37, 'feature_09'): 34.000000,
    (37, 'feature_10'): 4.000000,
    (37, 'feature_11'): 214.000000,
    (37, 'feature_12'): 0.759314,
    (37, 'feature_13'): 0.284057,
    (37, 'feature_14'): 0.417160,
    (37, 'feature_15'): -0.611075,
    (37, 'feature_16'): -0.513717,
    (37, 'feature_17'): -0.891423,
    (37, 'feature_18'): 1.849940,
    (37, 'feature_19'): 0.406756,
    (37, 'feature_20'): -1.608196,
    (37, 'feature_21'): -0.252663,
    (37, 'feature_22'): -0.271574,
    (37, 'feature_23'): -0.051405,
    (37, 'feature_24'): 0.098146,
    (37, 'feature_25'): -0.653961,
    (37, 'feature_26'): 0.173676,
    (37, 'feature_27'): -0.016497,
    (37, 'feature_28'): -0.404509,
    (37, 'feature_29'): -0.577262,
    (37, 'feature_30'): -0.731429,
    (37, 'feature_31'): -0.216460,
    (37, 'feature_32'): 3.018564,
    (37, 'feature_33'): -0.472061,
    (37, 'feature_34'): 3.139220,
    (37, 'feature_35'): 3.065858,
    (37, 'feature_36'): 0.842925,
    (37, 'feature_37'): 0.053283,
    (37, 'feature_38'): -0.074403,
    (37, 'feature_39'): 0.500129,
    (37, 'feature_40'): 0.082630,
    (37, 'feature_41'): 0.336223,
    (37, 'feature_42'): 0.643934,
    (37, 'feature_43'): -0.422367,
    (37, 'feature_44'): -0.418195,
    (37, 'feature_45'): 0.203037,
    (37, 'feature_46'): -0.702278,
    (37, 'feature_47'): 0.543305,
    (37, 'feature_48'): -0.195764,
    (37, 'feature_49'): 0.693364,
    (37, 'feature_50'): 0.953293,
    (37, 'feature_51'): 0.352567,
    (37, 'feature_52'): 0.471775,
    (37, 'feature_53'): 1.876459,
    (37, 'feature_54'): -0.143377,
    (37, 'feature_55'): 0.845516,
    (37, 'feature_56'): 0.301135,
    (37, 'feature_57'): -0.395703,
    (37, 'feature_58'): 0.738038,
    (37, 'feature_59'): -0.041240,
    (37, 'feature_60'): 1.270645,
    (37, 'feature_61'): -1.101531,
    (37, 'feature_62'): -0.358106,
    (37, 'feature_63'): -0.141883,
    (37, 'feature_64'): -0.255192,
    (37, 'feature_65'): 2.489247,
    (37, 'feature_66'): 0.537652,
    (37, 'feature_67'): 0.982107,
    (37, 'feature_68'): -0.158009,
    (37, 'feature_69'): 0.137389,
    (37, 'feature_70'): 0.478357,
    (37, 'feature_71'): 0.782692,
    (37, 'feature_72'): 0.581421,
    (37, 'feature_73'): -0.106056,
    (37, 'feature_74'): -0.111017,
    (37, 'feature_75'): 0.163867,
    (37, 'feature_76'): 0.169331,
    (37, 'feature_77'): -0.037563,
    (37, 'feature_78'): -0.029483,
    (38, 'feature_00'): 2.728506,
    (38, 'feature_01'): -0.745238,
    (38, 'feature_02'): 2.788789,
    (38, 'feature_03'): 2.343393,
    (38, 'feature_04'): 0.454731,
    (38, 'feature_05'): 0.862839,
    (38, 'feature_06'): 0.964795,
    (38, 'feature_07'): 2.089673,
    (38, 'feature_08'): 0.344931,
    (38, 'feature_09'): 50.000000,
    (38, 'feature_10'): 1.000000,
    (38, 'feature_11'): 522.000000,
    (38, 'feature_12'): 0.406531,
    (38, 'feature_13'): 0.618247,
    (38, 'feature_14'): 1.013270,
    (38, 'feature_15'): -0.952069,
    (38, 'feature_16'): -0.679168,
    (38, 'feature_17'): -0.597603,
    (38, 'feature_18'): 0.375125,
    (38, 'feature_19'): 1.975370,
    (38, 'feature_20'): -0.440974,
    (38, 'feature_21'): -0.072018,
    (38, 'feature_22'): 1.741353,
    (38, 'feature_23'): 1.380735,
    (38, 'feature_24'): -0.110494,
    (38, 'feature_25'): -0.874806,
    (38, 'feature_26'): 0.553424,
    (38, 'feature_27'): 0.532243,
    (38, 'feature_28'): 0.263214,
    (38, 'feature_29'): -0.757856,
    (38, 'feature_30'): -0.869204,
    (38, 'feature_31'): -0.062955,
    (38, 'feature_32'): 3.619233,
    (38, 'feature_33'): -0.386316,
    (38, 'feature_34'): 3.544560,
    (38, 'feature_35'): 3.120631,
    (38, 'feature_36'): -1.443649,
    (38, 'feature_37'): -0.257411,
    (38, 'feature_38'): -0.309567,
    (38, 'feature_39'): 1.366358,
    (38, 'feature_40'): -0.220885,
    (38, 'feature_41'): 0.029798,
    (38, 'feature_42'): 1.094489,
    (38, 'feature_43'): -0.051078,
    (38, 'feature_44'): -0.114243,
    (38, 'feature_45'): 0.517313,
    (38, 'feature_46'): 0.852201,
    (38, 'feature_47'): 0.522199,
    (38, 'feature_48'): -0.027275,
    (38, 'feature_49'): 0.471593,
    (38, 'feature_50'): 1.213111,
    (38, 'feature_51'): 0.263278,
    (38, 'feature_52'): 0.915804,
    (38, 'feature_53'): 1.862022,
    (38, 'feature_54'): 0.503819,
    (38, 'feature_55'): 1.310126,
    (38, 'feature_56'): 0.662521,
    (38, 'feature_57'): 1.654948,
    (38, 'feature_58'): 1.090367,
    (38, 'feature_59'): 0.535922,
    (38, 'feature_60'): 0.653011,
    (38, 'feature_61'): -1.101531,
    (38, 'feature_62'): -0.622853,
    (38, 'feature_63'): -0.363631,
    (38, 'feature_64'): -0.395652,
    (38, 'feature_65'): -0.016812,
    (38, 'feature_66'): 2.016734,
    (38, 'feature_67'): 0.241486,
    (38, 'feature_68'): 0.253229,
    (38, 'feature_69'): 0.228745,
    (38, 'feature_70'): 0.462717,
    (38, 'feature_71'): 0.799635,
    (38, 'feature_72'): 0.706102,
    (38, 'feature_73'): -0.376377,
    (38, 'feature_74'): -0.286764,
    (38, 'feature_75'): -0.359046,
    (38, 'feature_76'): -0.246135,
    (38, 'feature_77'): -0.288941,
    (38, 'feature_78'): -0.247774,
}


#### Global constants for feature processing

In [159]:
FEATURE_COLS = [f'feature_{i:02d}' for i in range(79)]
FEATURE_NULL_COLS = [f'{col}_is_null' for col in FEATURE_COLS]
MODEL_COLS = ['symbol_id', 'weight'] + FEATURE_COLS + FEATURE_NULL_COLS
FEATURE_TYPES = {
    # Int16 features
    'feature_11': pl.Int16,
    
    # Int8 features
    'feature_09': pl.Int8,
    'feature_10': pl.Int8,
    
    # All other features are Float32
    **{f'feature_{i:02d}': pl.Float32 for i in range(79) 
       if i not in [9, 10, 11]}
}

In [160]:
def preprocess_features(df: pl.DataFrame) -> np.ndarray:
    """Preprocess features and return numpy array ready for prediction"""
    global last_known_values
    
    # Extract all needed data as numpy arrays once
    symbol_ids = df['symbol_id'].to_numpy()
    weights = df['weight'].to_numpy()
    
    # Preallocate final array (features + null flags + weight + symbol_id)
    n_rows = len(df)
    processed = np.zeros((n_rows, len(MODEL_COLS)), dtype=np.float32)
    
    # Fill symbol_id and weight columns first
    processed[:, 0] = symbol_ids
    processed[:, 1] = weights
    
    # Process features and create null flags
    for i, col in enumerate(FEATURE_COLS):
        values = df[col].to_numpy()
        null_mask = np.isnan(values)
        
        # Fill null values
        if null_mask.any():
            for idx in np.where(null_mask)[0]:
                sym = symbol_ids[idx]
                values[idx] = last_known_values.get((sym, col),0)
        
        # Update last known values
        for sym, val in zip(symbol_ids[~null_mask], values[~null_mask]):
            last_known_values[(sym, col)] = val
            
        # Set feature values and null flags
        processed[:, 2+i] = values  # Features come after symbol_id and weight
        processed[:, 2+len(FEATURE_COLS)+i] = null_mask  # Null flags at the end
    
    return processed

In [178]:
# Global constants stay the same
# New globals for online learning - store complete daily data
online_data = {
   'daily_features': {},  # date_id -> list of feature batches for that day
   'daily_targets': {}    # date_id -> matched responder values for that day
}

In [182]:

def update_online_data(test: pl.DataFrame, lags: pl.DataFrame | None) -> None:
   """Store features and match with targets from lags"""
   global online_data
   
   current_date = test['date_id'][0]
   
   # Store feature batch
   if current_date not in online_data['daily_features']:
       online_data['daily_features'][current_date] = []
   online_data['daily_features'][current_date].append(test)
   
   # At start of new day (when we get lags), sort and store previous day's data
   if lags is not None and current_date > 0:
       prev_date = current_date - 1
       if prev_date in online_data['daily_features']:
           # Combine and sort all features from previous day
           all_features = pl.concat(online_data['daily_features'][prev_date])
           sorted_features = all_features.sort(['time_id', 'symbol_id'])
           
           # Sort lag data same way
           sorted_lags = lags.sort(['time_id', 'symbol_id'])
           
           # Store sorted features and corresponding responder values
           online_data['daily_features'][prev_date] = sorted_features
           online_data['daily_targets'][prev_date] = sorted_lags['responder_6_lag_1'].to_numpy()

def verify_reconstruction():
   """Verify our reconstruction matches test.parquet"""
   # Load test data which has ground truth
   test = pl.read_parquet(os.path.join(data_path, "test.parquet"))
   
   print("\nVerifying Data Reconstruction:")
   
   for date_id in sorted(online_data['daily_features'].keys()):
       if date_id not in online_data['daily_targets']:
           continue
           
       # Get ground truth data sorted same way
       actual = test.filter(pl.col('date_id') == date_id).sort(['time_id', 'symbol_id'])
       
       # Get our reconstructed data
       reconstructed_features = online_data['daily_features'][date_id]
       reconstructed_targets = online_data['daily_targets'][date_id]
       
       print(f"\nDate ID: {date_id}")
       
       # Verify features match
       for feature in [f'feature_{i:02d}' for i in range(79)]:
           features_match = np.array_equal(
               reconstructed_features[feature].to_numpy(),
               actual[feature].to_numpy(),
               equal_nan=True
           )
           if not features_match:
               print(f"Feature mismatch in {feature}")
               return
       
       # Verify targets match
       targets_match = np.array_equal(
           reconstructed_targets,
           actual['responder_6'].to_numpy(),
           equal_nan=True
       )
       
       print("Features match: True")
       print(f"Targets match: {targets_match}")
       
       if not targets_match:
           print("\nFirst few rows:")
           print("Reconstructed targets:", reconstructed_targets[:5])
           print("Actual targets:", actual['responder_6'].head(5))

### Load model

In [163]:
import lightgbm as lgb

import torch
import torch.nn as nn

In [164]:
# Some models
# last_750.txt
# last_1000.txt
# last_1250.txt

# Define models and their weights for ensemble
MODEL_CONFIGS = [
    # {"name": "last_750.txt", "weight": 0.25, "type": "lgb"},
    # {"name": "last_1000.txt", "weight": 0.25, "type": "lgb"},
    # {"name": "last_1250.txt", "weight": 0.25, "type": "lgb"},

    {"name": "mlp_test_1.pt", "weight": 1, "type": "mlp"}
]

In [165]:
class MLPModel(nn.Module):
    def __init__(self, input_dim: int):
        super().__init__()
        self.input_dim = input_dim
        self.hidden_dims = []  # Will be set from metadata
        self.dropout = 0.1     # Will be set from metadata
        self.layers = None     # Will be built when dimensions are set
        
    def build_layers(self):
        self.layers = nn.Sequential(
            nn.Linear(self.input_dim, self.hidden_dims[0]),
            nn.ReLU(),
            nn.Dropout(self.dropout),
            *[nn.Sequential(
                nn.Linear(self.hidden_dims[i], self.hidden_dims[i+1]),
                nn.ReLU(),
                nn.Dropout(self.dropout)
            ) for i in range(len(self.hidden_dims)-1)],
            nn.Linear(self.hidden_dims[-1], 1)
        )
        
    def forward(self, x):
        return self.layers(x).squeeze()

In [166]:
def setup_prediction():
    """Load and setup models for prediction"""
    models = []
    for config in MODEL_CONFIGS:
        if IS_KAGGLE:
            model_path = os.path.join(trained_models_path, config["name"])
            metadata_path = os.path.join(trained_models_path, os.path.splitext(config["name"])[0] + "_metadata.json")
        else:
            model_path = os.path.join(PROJ_DIR, "trained_models", config["name"])
            metadata_path = os.path.join(PROJ_DIR, "trained_models", os.path.splitext(config["name"])[0] + "_metadata.json")
        
        if config["type"] == "lgb":
            # Load LightGBM model
            models.append({"model": lgb.Booster(model_file=model_path), "type": "lgb"})
        else:
            # Load metadata for MLP configuration
            with open(metadata_path, 'r') as f:
                metadata = json.load(f)
            
            # Create MLP with correct architecture from metadata
            mlp_model = MLPModel(input_dim=len(MODEL_COLS))
            mlp_model.hidden_dims = metadata["model_params"]["hidden_dims"]
            mlp_model.dropout = metadata["model_params"]["dropout"]
            mlp_model.build_layers()  # Build layers before loading state dict
            
            mlp_model.load_state_dict(torch.load(model_path))
            mlp_model.eval()  # Set to evaluation mode
            mlp_model = mlp_model.to('cuda' if torch.cuda.is_available() else 'cpu')
            models.append({"model": mlp_model, "type": "mlp"})
    
    return models
    
# Load models globally
models = setup_prediction()

/tmp/ipykernel_205645/4088639429.py:26: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  mlp_model.load_state_dict(torch.load(model_path))


### Predict API

In [176]:
# TODO: FLoat value in predictions? Type 64?

# Global variables
lags_ : pl.DataFrame | None = None

def predict(test: pl.DataFrame, lags: pl.DataFrame | None) -> pl.DataFrame | pd.DataFrame:
    """Make a prediction."""
    global lags_
    
    if lags is not None:
        lags_ = lags
        if not IS_KAGGLE:
            progress_bar.update(1)
    
    # Update stored data for online learning
    update_online_data(test, lags)

    # Preprocess and select needed columns
    processed = preprocess_features(test)
 
    # Convert to numpy and predict
    all_predictions = []
    for model_dict in models:
        if model_dict["type"] == "lgb":
            all_predictions.append(model_dict["model"].predict(processed))
        else:
            with torch.no_grad():
                tensor_input = torch.FloatTensor(processed).to('cuda' if torch.cuda.is_available() else 'cpu')
                pred = model_dict["model"](tensor_input).cpu().numpy()
                all_predictions.append(pred)
                
    predictions = np.average(all_predictions, axis=0, weights=[c["weight"] for c in MODEL_CONFIGS])

    # Clip them between possible responder values
    predictions = np.clip(predictions, a_min = -5, a_max = 5)
    
    # Create return DataFrame efficiently
    predictions = test.select('row_id').with_columns(
        pl.Series('responder_6', predictions, dtype=pl.Float64)
    )
    
    assert isinstance(predictions, pl.DataFrame | pd.DataFrame) # The predict function must return a DataFrame
    assert predictions.columns == ['row_id', 'responder_6']     # with columns 'row_id', 'responer_6'
    assert len(predictions) == len(test)                        # and as many rows as the test data.

    return predictions

### Submit

In [183]:
# Set up inference server
inference_server = kaggle_evaluation.jane_street_inference_server.JSInferenceServer(predict)

# Run based on environment
if os.getenv('KAGGLE_IS_COMPETITION_RERUN'):
    inference_server.serve()
else:    
    inference_server.run_local_gateway(
        (
            os.path.join(data_path, "test.parquet"),
            os.path.join(data_path, "lags.parquet"),
        )
    )

Processing days: 12it [05:45, 24.10s/it]

Processing days: 13it [05:57, 20.34s/it]

GatewayRuntimeError: (<GatewayRuntimeErrorType.SERVER_RAISED_EXCEPTION: 3>, "'DataFrame' object has no attribute 'append'")

In [181]:
verify_reconstruction()


Verifying Data Reconstruction:

Date ID: 1
Feature mismatch in feature_00


### Local Scoring

In [ ]:
if not IS_KAGGLE:
    # Close progress bar if it exists
    if progress_bar is not None:
        progress_bar.close()

    # Local scoring
    print("Scoring submission...")
    predictions = pl.read_parquet("submission.parquet").sort('row_id')  # Sort by row_id
    test_data = pl.read_parquet(os.path.join(data_path, "test.parquet")).sort('row_id')  # Sort by row_id  

    # Score only rows marked for scoring
    mask = test_data['is_scored']
    score = r2_score_weighted(
        test_data.filter(mask)['responder_6'].to_numpy(),
        predictions.filter(mask)['responder_6'].to_numpy(),
        test_data.filter(mask)['weight'].to_numpy()
    )
    print(f"Overall R² score: {score}")

In [ ]:
# Pre, clip: 0.007479486611981945, 12:11<00:00, 14.64s/it]
# Post clip: 0.007480604868541518, [10:20<00:00, 12.42s/it]

# Wth Clip: Overall R² score: 0.009906675990193792
# No clip: Overall R² score:  0.009906675990193792
# Clip (-2 to 2): Overall R²  0.009906675990193792
# Clip (-1, to 1) l R² score: 0.009902191264148152
